In [1]:
from time import *
from itertools import combinations
from CGRtools import smiles, ReactionContainer
# from rdkit import Chem

class MolGraph:
    
    # graph_data の要件
    # 原子情報、結合情報 2つのタプル
    # 原子情報 = (非水素原子に対する) 原子index(1スタート以外も可), 原子番号, 付加水素数
    # 結合情報 = 結合index, 原子indexの組, 結合次数(ケクレ化後の整数値に限る)
    
    def __init__( self, strn, mode='CGRtools' ): # CGRtools dependency
        
        t = time()
        
        def graphdata_by_CGRtools( strn ):
            mol = smiles( strn )
            mol.kekule()
            mol_Hs = deepcopy( mol )
            mol_Hs.explicify_hydrogens()
            an_Hs = { idx:at.atomic_number for idx,at in mol_Hs.atoms() }
            #old# Hs = { k:sum( [ 1 for i,j,_ in mol_Hs.bonds() if (an_Hs[i]==1 or an_Hs[j]==1) and (i==k or j==k)] ) for k,_ in mol_Hs.atoms() if an_Hs[k]!=1 }
            Hs = { k:sum( [ 1 for i,j,_ in mol_Hs.bonds() if (an_Hs[i]==1 or an_Hs[j]==1) and (i==k or j==k)] ) for k,_ in mol_Hs.atoms() }
            
            atom_info = [ (x,y.atomic_number,Hs[x]) for x,y in mol.atoms() ]
            
            bond_info = []
            for i, (x, y, z) in enumerate( mol.bonds() ):
                if x > y:
                    x, y = y, x
                bond_info.append((i, (x, y), z.order))
            # bond_info = [ (i,(x,y),z.order) for i, (x,y,z) in enumerate( mol.bonds() ) ]
            
            return atom_info, bond_info
        
        '''
        def graphdata_by_RDKit( strn, mode ):
            if mode == 'inchi' or mode == 'InChI':
                mol = Chem.MolFromInchi( strn )
            if mode == 'smiles' or mode == 'SMILES':
                mol = Chem.MolFromSmiles( strn )
            
            Chem.Kekulize( mol )
            mol_Hs = Chem.AddHs( mol )
            an = { at.GetIdx():at.GetAtomicNum() for at in mol_Hs.GetAtoms() }
            bonds = [ (bn.GetBeginAtomIdx(), bn.GetEndAtomIdx()) for bn in mol_Hs.GetBonds() ]
            Hs = { k:sum( [ 1 for i,j in bonds if (an[i]==1 or an[j]==1) and (i==k or j==k)] ) for k in an.keys() if an[k]!=1 }
            
            atom_info = [ (at.GetIdx(), at.GetAtomicNum(), Hs[at.GetIdx()]) for at in mol.GetAtoms() ]
            # bond_info = [ (i, (bn.GetBeginAtomIdx(),bn.GetEndAtomIdx()),int(bn.GetBondTypeAsDouble())) for i, bn in enumerate( mol.GetBonds() ) ]
            bond_info = []
            for i, bn in enumerate( mol.GetBonds() ):
                x, y = bn.GetBeginAtomIdx(), bn.GetEndAtomIdx()
                if x > y:
                    x, y = y, x
                bond_info.append((i, (x, y), int(bn.GetBondTypeAsDouble())))
            
            return atom_info, bond_info
        '''
        
        
        if mode == 'CGRtools':
            atom_info, bond_info = graphdata_by_CGRtools( strn )
            self.MOL = smiles( strn )
            self.MOL.clean2d()
        
        # if mode == 'RDkit':
        #    atom_info, bond_info = graphdata_by_RDKit( strn, 'smiles' )
        
        # 原子ラベル(1から開始、描画のラベルと揃えるため) → 原子番号
        self.atoms   = [ idx for idx,_,_ in atom_info ]
        self.atoms.sort() # SMARTS 時に重要
        self.an      = { idx:an for idx,an,_ in atom_info }
        
        # 結合ラベル(0から開始、リストを利用、必ず原子ラベル昇順) → 原子ラベル, 原子番号, 結合次数, 結合種
        self.bonds = [ pair for _,pair,_ in bond_info ]
        self.bn_an = [ (self.an[x],self.an[y]) for x,y in self.bonds ]
        self.order = [ z for _,_,z in bond_info ]
        self.bn_kind = [ (x,y) for x,y in self.bn_an if x <= y ] + [ (y,x) for x,y in self.bn_an if x > y ]
        
        # 水素置換数
        self.dic_hyd = { idx:Hs for idx,_,Hs in atom_info }
        
        # 近傍リストの辞書
        self.dic_neib = dict()
        for n in self.atoms:
            n_neib = []
            for x, y in self.bonds:
                if n==x:
                    n_neib.append( y )
                    continue
                if n==y:
                    n_neib.append( x )
            
            self.dic_neib[n] = n_neib
        
        # 結合次数の辞書
        self.dic_order = dict()
        for x, y in combinations( self.atoms, 2 ):
            if x > y:
                x, y = y, x
            if not ( x, y ) in self.bonds:
                self.dic_order[(x,y)] = 0
                self.dic_order[(y,x)] = 0
            else:
                idx = self.bonds.index( ( x, y ) )
                self.dic_order[(x,y)] = self.order[idx]
                self.dic_order[(y,x)] = self.order[idx]
        
    #--------------------------------------------------------------------------------------------------------------
    # 2つの原子ラベル → 結合の有無
    def refer_bond( self, n, m ):
        if n > m:
            n, m = m, n
        
        if (n, m) in self.bonds:
            b_num = self.bonds.index((n, m))
            return self.bn_kind[b_num]
        
        return None
    
    def refer_bond_order( self, n, m ):
        if n > m:
            n, m = m, n
        
        if (n,m) in self.bonds:
            ind = self.bonds.index((n, m))
            return self.order[ind]
        else:
            return 0
    
    # 2つの結合ラベル → つながる原子の原子番号 例: C1-C2 と C1-O3 → 6
    def bond_common_atom( self, n, m ):
        n1, n2 = self.bonds[n]
        m1, m2 = self.bonds[m]
        if n1==m1 or n1==m2:
            return self.an[n1]
        if n2==m1 or n2==m2:
            return self.an[n2]
        return None
    
    # 2つの結合ラベルから連続する3原子ラベルを返す 1-3,2-1 → 2-1-2 前半と後半は混ぜない
    # n1,n2 を第3成分にしたり、m1,m2を第2成分にしてはいけない
    def bond_mapping( self, n, m ):
        n1, n2 = self.bonds[n]
        m1, m2 = self.bonds[m]
        # A-B, A-C -> B,A,C
        if n1==m1:
            return n2, n1, m2
        # A-B, C-A -> B,A,C
        if n1==m2:
            return n2, n1, m1
        # A-B, B-C -> A,B,C
        if n2==m1:
            return n1, n2, m2
        # A-B, C-B -> A,B,C
        if n2==m2:
            return n1, n2, m1
        return None

    '''
    # ========== k-neighbour functions =================================================================================================
    def bfs( self, n, k ):
        
        dic = deepcopy( self.dic_neib )
        result = []
        neibs = [ ( n, self.dic_neib[n] ) ]
        result += [neibs]
        visited = {n} | set( self.dic_neib[n] ) # 訪問済み頂点
        
        if k==0:
            return [n]
        if k==1:
            return neibs
        
        while k>1:
            next_neib = []
            #近傍の探索
            for center, neib in neibs:
                for x in neib:
                    if x>0:
                        next_neib.append( (x, [y for y in self.dic_neib[x] if y!=center]) )
            
            # サイクル検出と書き換え
            next_neib2 = []
            for x, ls in next_neib:
                ls2 = [ z for z in ls if not z in visited ] + [ -z for z in ls if z in visited ]
                next_neib2.append( (x,ls2) )
            
            #探索済みノードの追加
            for x, ls in next_neib:
                visited |= set( ls )
            
            if next_neib2 == []:
                break
            result += [next_neib2]
            neibs = next_neib2
            k -= 1
            
        return result
    
    # ----------------------------------------------------------------------------------------------------------------
    
    def merge_sequence( self, xs ):
        
        def merge( p_nodes, c_nodes ):
            for n, p in p_nodes:
                for m, c in c_nodes:
                    if (m in p) and (not n in c) and (not -n in c):
                        p.pop( p.index(m) )
                        if c == []:
                            p.append( m )
                        else:
                            p.append( [m,c] )
            return p_nodes
        
        # reverse, merge で破壊的変更を行うのでdeepcopy
        xs = deepcopy( xs )
        
        if len(xs)==1:
            return xs[0]
        
        xs.reverse()
        
        for i, x in enumerate( xs ):
            if xs[i] == xs[-1]:
                break;
            merge( xs[i+1], xs[i] )
        
        xs1, xs2 = xs[-1][0]
        
        return [xs1, xs2]
    
    # ----------------------------------------------------------------------------------------------------------------
    def convert_AN( self, x ):
        
        def calc_nest( ls ):
            if ls==[]:
                return 0
            if type( ls ) is int:
                return 0
            tmp = []
            for x in ls:
                tmp.append( 1 + calc_nest( x ) )
            return max( tmp )
        
        out = []
        if type( x ) is int:
            if x>0:
                return self.an[x] 
            else:
                return -self.an[-x]
        else:
            conv_list = [ self.convert_AN( y ) for y in x ]
            
            # 入れ子階層がズレている場合は階層数でソート
            try:
                conv_list.sort()
            except TypeError:
                conv_list = sorted( conv_list, key=calc_nest )
                
            out += conv_list
        return out
    
    def convert_AN_with_Hs( self, x ):
        
        def calc_nest( ls ):
            if ls==[]:
                return 0
            if type( ls ) is int:
                return 0
            tmp = []
            for x in ls:
                tmp.append( 1 + calc_nest( x ) )
            return max( tmp )
        
        out = []
        if type( x ) is int:
            if x>0:
                return self.an[x], self.dic_hyd[x]
            else:
                return -self.an[-x], self.dic_hyd[-x]
        else:
            conv_list = [ self.convert_AN_with_Hs( y ) for y in x ]
            
            # 入れ子階層がズレている場合は階層数でソート
            try:
                conv_list.sort()
            except TypeError:
                conv_list = sorted( conv_list, key=calc_nest )
                
            out += conv_list
        return out
    
    # ----------------------------------------------------------------------------------------------------------------
    def get_k_spanning_tree( self, n, k ):
        ms = self.merge_sequence( self.bfs( n, k ) )
        return self.convert_AN( ms )
    
    def all_k_spanning( self, k ):
        return [ self.get_k_spanning_tree( x, k ) for x in self.atoms ]
    
    def get_k_spanning_tree_with_Hs( self, n, k ):
        ms = self.merge_sequence( self.bfs( n, k ) )
        return self.convert_AN_with_Hs( ms )
    
    def all_k_spanning_with_Hs( self, k ):
        return [ self.get_k_spanning_tree_with_Hs( x, k ) for x in self.atoms ]
    
    # -----　path enumeration ----------------------------------------------------------------------------------------
    def all_atom_pathes( self, n, m ):
        
        # add of only next neighbors to a given path
        def get_next_pathes( ls ):
            
            neib = self.dic_neib[ ls[-1] ]
            pathes = []
            for x in neib:
                # limited to a simple path
                if ls[-2] != x and not x in ls:
                    pathes.append( ls+[x] )
            
            return pathes
        
        neib = self.dic_neib[n]
        if neib == []:
            return []
        
        res = []
        pathes = []
        for x in neib:
            pathes.append( [n,x] )
            if x==m:
                res.append( [n,x] )
        
        while pathes != []:
            next_pts = []
            for pt in pathes:
                next_pts += get_next_pathes( pt )
            pathes = next_pts
            for x in pathes:
                if x[-1] == m:
                    res.append( x )
        
        return res
        '''

In [2]:
from copy import deepcopy
from mset import mset
from cliques import largest_cliques, maximal_cliques, largest_cliques_limited, maximal_cliques_limited
import igraph as ig
from itertools import permutations

class ReactionMap:
    
    def __init__( self, str_smiles, mode='CGRtools' ):
        
        t = time()
        print( 'target reaction:', str_smiles )
        sm_rct, sm_prd = str_smiles.split('>>')
        
        self.rct = MolGraph( sm_rct, mode )
        self.prd = MolGraph( sm_prd, mode )
        
        if mode == 'CGRtools':
            self.RCT = self.rct.MOL
            self.PRD = self.prd.MOL
            self.RXN = ReactionContainer( [self.RCT], [self.PRD] )
            self.RXN.clean2d()
        
        print( '# initialization: ' )
        print( '\t>> time: ', str(round( time() - t, 6)).rjust(4), 'sec' )
        
    def unmapped_area( self, mp ):
        
        if mp == []:
            return [], []
        
        r_mapped, p_mapped = zip( *mp )
        r_unmap = [ x for x in self.rct.atoms if not x in r_mapped ]
        p_unmap = [ x for x in self.prd.atoms if not x in p_mapped ]
        
        return r_unmap, p_unmap
    
    # モジュラー積を返却
    def form_graph_product( self, mode='bond', neib=0, premap=[] ):
        
        t = time()
        
        r_area, p_area = self.unmapped_area( premap )
        
        #r_neib = self.rct.all_k_spanning( neib )
        #p_neib = self.prd.all_k_spanning( neib )
        r_neib = list( self.rct.an.values() )
        p_neib = list( self.prd.an.values() )
        
        if mode == 'atom':
            
            nodes = []
            for i, x in enumerate( r_neib ):
                for j, y in enumerate( p_neib ):
                    if x == y:
                        if r_area == [] and p_area == []: 
                            nodes.append((i+1, j+1))
                        else:
                            if (i+1 in r_area) and (j+1 in p_area):
                                nodes.append((i+1, j+1))
            
            #if len( nodes ) > 500:
            #    print( len( nodes ), 'too many nodes' )
            #    return 'too large', None
            
            edges = []
            for node1, node2 in combinations( nodes, 2 ):
                r1, p1 = node1
                r2, p2 = node2
                if r1!=r2 and p1!=p2:
                    r_bond = self.rct.refer_bond( r1, r2 )
                    p_bond = self.prd.refer_bond( p1, p2 )
                    i = nodes.index( node1 )
                    j = nodes.index( node2 )
                    if r_bond == p_bond:
                        edges.append((i, j))
        
        if mode == 'bond':
            
            rb_neib = [ (r_neib[x-1], r_neib[y-1]) for x,y in self.rct.bonds ]
            pb_neib = [ (p_neib[x-1], p_neib[y-1]) for x,y in self.prd.bonds ]
            
            nodes = []
            for i, (x,y) in enumerate( rb_neib ):
                for j, (v,w) in enumerate( pb_neib ):
                    # 結合のn近傍
                    if (x==v and y==w) or (x==w and y==v):
                        if r_area == [] and p_area == []:
                            nodes.append((i,j))
                        else:
                            r1, r2 = self.rct.bonds[i]
                            p1, p2 = self.prd.bonds[j]
                            #if ((r1 in r_area) or (r2 in r_area)) and ((p1 in p_area) or (p2 in p_area)):
                            if (r1 in r_area) and (r2 in r_area) and (p1 in p_area) and (p2 in p_area):
                                nodes.append((i,j))
            
            #if len( nodes ) > 500:
            #    print( len( nodes ), 'too many nodes' )
            #    return 'too large', None
            
            edges = []
            for node1, node2 in combinations( nodes, 2 ):
                r1, p1 = node1
                r2, p2 = node2
                if r1!=r2 and p1!=p2:
                    # 共通原子があるか、ないか
                    if self.rct.bond_common_atom( r1, r2 ) == self.prd.bond_common_atom( p1, p2 ):
                        i = nodes.index( node1 )
                        j = nodes.index( node2 )
                        edges.append( ( i, j ) )
        
        #print( '# calculation of product graph: mode =', mode, ', neib =', neib )
        #print( '\t>> '+str(mode)+'-to-'+str(mode)+' product graph:', str( len( nodes ) ).rjust(8), 'nodes,', str( len( edges ) ).rjust(8), 'edges,' )
        #print( '\t>> time: ', str( round( time() - t, 6 )).rjust(2), 'sec' )
        
        return mode, nodes, edges
    
    # 結合マップ → 原子マップ
    def to_atom_mapping( self, bond_map ):
        
        atom_map = []
        
        # 異種元素結合の対応
        for n, m in bond_map:
            r1, r2 = self.rct.bonds[n]
            p1, p2 = self.prd.bonds[m]
            r_an1, r_an2 = self.rct.bn_an[n]
            p_an1, p_an2 = self.prd.bn_an[m]
            
            # 異種元素間結合で
            if r_an1!=r_an2 and p_an1!=p_an2:
                # A-B と C-D が A->C かつ B->D のとき
                if r_an1==p_an1 and r_an2==p_an2:
                    atom_map += [(r1,p1), (r2,p2)]
                #  A-B と C-D が A->D かつ B->C のとき
                if r_an1==p_an2 and r_an2==p_an1:
                    atom_map += [(r1,p2), (r2,p1)]
        
        # 同種元素結合の対応
        for (r1,p1), (r2,p2) in combinations( bond_map, 2 ):
            if self.rct.bond_common_atom( r1, r2 ):
                
                # 結合の r1→p1 / r2→p2 の 対応は確定しているので
                # r1 の中身が p2 の中身に対応することは決してない
                # bond_mapping は その対応関係は保存している
                rx, ry, rz = self.rct.bond_mapping( r1, r2 )
                px, py, pz = self.prd.bond_mapping( p1, p2 )
                atom_map += [ (rx, px), (ry, py), (rz, pz)]
        
        # 重複をのぞく
        atom_map = list( set( atom_map ) )
        atom_map.sort()
        
        return atom_map
    
    # 最大クリークを求めて原子マップを得る
    def maximum_cliques( self, modular, premap=[], completion=True, optimizer='aki', timeout=10.0 ):
        
        t = time()
        
        mode, nodes, edges = modular
        
        if optimizer == 'aki':
            #maximum_cliques = largest_cliques( mapdat.edges )
            max_cqs = largest_cliques_limited( edges, timeout )
            
        if optimizer == 'igraph':
            max_cqs = ig.Graph( edges ).largest_cliques()
        
        if max_cqs == []:
            print('! ----- empty cliques ----- !')
            total_maps = [premap]
        else:
            maps = []
            for cq in max_cqs:
                mapping = [ nodes[x] for x in cq ]
                mapping.sort()
                maps.append( mapping )
            
            if mode == 'atom':
                atom_maps = maps
            
            if mode == 'bond':
                #print( 'transformation from bond-to-bond to atom-to-atom mapping' )
                atom_maps = []
                for bond_map in maps:
                    atom_map = self.to_atom_mapping( bond_map )
                    atom_map.sort()
                    atom_maps.append( atom_map )
            
            total_maps = [ list( set( x + premap ) ) for x in atom_maps ]
        
        # ユニークに定まるものを追加
        # 順列網羅
        '''
        if completion == True:
            
            total_added = []
            for x in total_maps: 
                
                r_mapped, p_mapped = zip( *x )
                r_rsd = [ x for x in self.rct.atoms if not x in r_mapped ]
                p_rsd = [ x for x in self.prd.atoms if not x in p_mapped ]
                
                if len( r_rsd ) >= 6:
                    continue
                
                prd_an = [ self.prd.an[x] for x in p_rsd ]
                
                for perm in list( permutations( r_rsd ) ):
                    rct_an = [ self.rct.an[x] for x in perm ]
                    
                    if rct_an == prd_an:
                        additional = list( zip( perm, p_rsd ) )
                        #print( 'permutation', perm, '--->', p_rsd )
                        total_added.append( x + additional )
            
            if total_added != []:
                total_maps = total_added
        
        for x in total_maps:
            x.sort()
        '''
        
        running_time = time() - t
        
        print( '# maximum cliques: mode =', completion, ', timeout =', timeout )
        if total_maps != []:
            print( '\t>> ',len(total_maps[0]), '/', len( self.rct.atoms ), 'atoms were mapping..' )
        print( '\t>> ', str(len(nodes)).rjust(8), 'atoms,', str(len(edges)).rjust(8), 'nodes,' )
        print( '\t>> time: ', str( round( running_time, 6) ).rjust(4), 'sec' )
        
        if running_time + 0.01 > timeout:
            print( '\t>> QUALITY = APPROX' )
        
        else:
            print( '\t>> QUALITY = EXACT' )
        
        return total_maps
    
    def permutation_complete( self, premaps=[] ):
        
        total_added = []
        for x in premaps: 
        
            r_mapped, p_mapped = zip( *x )
            r_rsd = [ x for x in self.rct.atoms if not x in r_mapped ]
            p_rsd = [ x for x in self.prd.atoms if not x in p_mapped ]
            
            # if len( r_rsd ) >= 7:
            #     print( 'too large permutation was detected: ', len( r_rsd ), '!' )
            #     raise Exception    
                    
            prd_an = [ self.prd.an[x] for x in p_rsd ]
            
            for perm in list( permutations( r_rsd ) ):
                rct_an = [ self.rct.an[x] for x in perm ]
                
                if rct_an == prd_an:
                    additional = list( zip( perm, p_rsd ) )
                    total_added.append( x + additional )
        
        return total_added
    
    '''
    # ユニークペアを探す計算
    def find_unique_pair( self ):
        
        t = time()
        
        unique_pair = []
        neib_list = [ 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 ]
        history = []

        for z in neib_list:
            # 多重集合クラスで重複しない分子サイトを探す
            cand_r = []
            r_tree = self.rct.all_k_spanning_with_Hs( z )
            r_tree_ms = mset( r_tree )
            r_unique = [ (r_tree.index(x)+1,x) for x,y in r_tree_ms.tp if y==1 ]
            
            cand_p = []
            p_tree = self.prd.all_k_spanning_with_Hs( z )
            p_tree_ms = mset( p_tree )
            p_unique = [ (p_tree.index(x)+1,x) for x,y in p_tree_ms.tp if y==1 ]
            
            # 重複しないもので一致したらユニークペアに
            for i, x in r_unique:
                for j, y in p_unique:
                    if x == y and not (i,j) in unique_pair:
                        unique_pair.append( ( i, j ) )
            
            history.append( len( unique_pair ) )
            
            # 3連続で登録ペアが変化しなければ終了
            if len( history ) >= 3:
                if history[-1]!=0 and history[-1] == history[-2] and history[-1]==history[-3]:
                    break
            
        # 真の孤立点 重複していても割り振れる
        isol_pair = []
        r_isol = [ i+1 for i, x in enumerate( self.rct.dic_neib.values() ) if x==[] ]
        p_isol = [ i+1 for i, x in enumerate( self.prd.dic_neib.values() ) if x==[] ]
        #print( 'isolated atoms', r_isol, p_isol )
        cp_isol = deepcopy( p_isol )
        for x in r_isol:
            for y in cp_isol:
                if self.rct.an[x] == self.prd.an[y]:
                    isol_pair.append((x,y))
                    cp_isol.remove( y )
                    break
        unique_pair += isol_pair
        
        # unique_pair による疑似孤立点
        r_unique = [ x for x,y in unique_pair ]
        p_unique = [ y for x,y in unique_pair ]
        r_isol = []
        p_isol = []
        for x in [ x for x in self.rct.atoms if not x in r_unique ]:
            if [ x for x in self.rct.dic_neib[x] if not x in r_unique ] == []:
                r_isol.append( x )
        for x in [ x for x in self.prd.atoms if not x in p_unique ]:
            if [ x for x in self.prd.dic_neib[x] if not x in p_unique ] == []:
                p_isol.append( x )
        #print( 'pseudo isolated atoms', r_isol, p_isol )
        
        
        #cp_isol = deepcopy( p_isol )
        #for x in r_isol:
        #    for y in cp_isol:
        #        r_neib = self.rct.dic_neib[x]
        #        p_neib = self.prd.dic_neib[y]
        #        if r_neib == [] or p_neib == []:
        #            break
        #        r_atom = r_neib[0]
        #        p_atom = p_neib[0]
        #        # 相手が一意に定まっていれば、結合次数の一致までみてリストに追加
        #        if len( r_neib ) == 1 and self.rct.an[r_atom] == self.prd.an[p_atom]:
        #            if x < r_atom:
        #                ind_r = self.rct.bonds.index( (x,r_atom) )
        #            else:
        #                ind_r = self.rct.bonds.index( (r_atom,x) )
        #            if y < p_atom:
        #                ind_p = self.prd.bonds.index( (y,p_atom) )
        #            else:
        #                ind_p = self.prd.bonds.index( (p_atom,y) )
        #            if self.rct.order[ind_r] == self.prd.order[ind_p]:
        #                unique_pair.append( ( x, y ) )
        #                cp_isol.remove( y )
        #                break
        
        unique_pair.sort()
        
        print( '# find unique pair' )
        print( '\t>>', len( unique_pair ), 'pairs were found..' )
        print( '\t>>', unique_pair )
        print( '\t>> time: ', str( round( time() - t, 6 ) ).rjust(4), 'sec' )
        
        return unique_pair
        '''
    
    
    def isomorphism( self, maps, mode='bond' ):
        
        t = time()
        
        if len( maps ) > 10000:
            print( 'isomorphism is skipped because of too many', len( maps ), 'mappings' )
            return [ set( i for i,_ in enumerate( maps ) ) ]
            
        r_iso = []
        p_iso = []
        
        if mode == 'skelton':
            for mp in maps:
                
                r_mapped, p_mapped = zip( *mp )
                mapped_r_bonds = [ (x,y) for x, y in self.rct.bonds if x in r_mapped and y in r_mapped ]
                mapped_p_bonds = [ (x,y) for x, y in self.prd.bonds if x in p_mapped and y in p_mapped ]
                to_rct = { y:x for x, y in mp }
                to_prd = { x:y for x, y in mp }
                conv_p_bonds = []
                conv_r_bonds = []
                
                for x, y in mapped_p_bonds:
                    s, t = to_rct[x], to_rct[y]
                    if s > t:
                        s, t = t, s
                    conv_p_bonds.append(( s, t ))
                
                for x, y in mapped_r_bonds:
                    s, t = to_prd[x], to_prd[y]
                    if s > t:
                        s, t = t, s
                    conv_r_bonds.append(( s, t ))
                
                conv_r_bonds.sort()
                conv_p_bonds.sort()
                
                if mode == 'skelton':
                    rs, ps = zip( *mp )
                    r_Hs = [ ( to_prd[x], self.rct.dic_hyd[x]) for x in rs ]
                    p_Hs = [ ( to_rct[x], self.prd.dic_hyd[x]) for x in ps ]
                    r_Hs.sort()
                    p_Hs.sort()
                    r_iso.append(( conv_r_bonds, r_Hs ))
                    p_iso.append(( conv_p_bonds, p_Hs ))
        
        if mode == 'bond':
            for mp in maps:
                
                r_mapped, p_mapped = zip( *mp )
                mapped_r_bonds = [ (x, y, self.rct.dic_order[(x, y)]) for x, y in self.rct.bonds if x in r_mapped and y in r_mapped ]
                mapped_p_bonds = [ (x, y, self.prd.dic_order[(x, y)]) for x, y in self.prd.bonds if x in p_mapped and y in p_mapped ]
                to_rct = { y:x for x, y in mp }
                to_prd = { x:y for x, y in mp }
                conv_p_bonds = []
                conv_r_bonds = []
                
                for x, y, order in mapped_p_bonds:
                    s, t = to_rct[x], to_rct[y]
                    if s > t:
                        s, t = t, s
                    conv_p_bonds.append(( s, t, order ))
                
                for x, y, order in mapped_r_bonds:
                    s, t = to_prd[x], to_prd[y]
                    if s > t:
                        s, t = t, s
                    conv_r_bonds.append(( s, t, order ))
                
                rs, ps = zip( *mp )
                r_Hs = [ ( to_prd[x], self.rct.dic_hyd[x]) for x in rs ]
                p_Hs = [ ( to_rct[x], self.prd.dic_hyd[x]) for x in ps ]
                r_Hs.sort()
                p_Hs.sort()
                conv_r_bonds.sort()
                conv_p_bonds.sort()
                
                r_iso.append(( conv_r_bonds, r_Hs ))
                p_iso.append(( conv_p_bonds, p_Hs ))
        
        # 対称性ネットワークの解析
        # マップ間ネットワークの隣接リストを書く
        connectivity = []
        for i, j in combinations( [ k for k, _ in enumerate( maps ) ], 2 ):
            if r_iso[i] == r_iso[j] or p_iso[i] == p_iso[j]:
                connectivity.append(( i, j ))
        
        maps = [ k for k, _ in enumerate( maps ) ]
        #print( maps, connectivity )
        
        return connected_comp( maps, connectivity )
    
    
    # 多重結合の変化は検出しない
    def change_bonds( self, mp ):
        
        print( mp )
        dic_map = dict( mp )
        cleav_bonds = []
        for x, y in self.rct.bonds:
            if (not x in dic_map.keys()) or (not y in dic_map.keys()):
                continue
            if self.prd.refer_bond( dic_map[x], dic_map[y] ) == None:
                cleav_bonds.append( ( x, y ) )
        
        dic_map_rev = dict( [ (y,x) for x,y in mp ] )
        form_bonds = []
        for x,y in self.prd.bonds:
            if (not x in dic_map_rev.keys()) or (not y in dic_map_rev.keys()):
                continue
            if self.rct.refer_bond( dic_map_rev[x], dic_map_rev[y] ) == None:
                form_bonds.append( ( x, y ) )
        
        #print( 'cleavage', cleav_bonds )
        #print( 'formation', form_bonds )
        return cleav_bonds, form_bonds
        
    # スコアリング関数
    # todo 他のスコア計算の実装
    def score( self, maps, mode='hydrogen' ):
        
        t = time()
        
        scores = []
        for mp in maps:
            sc = 0
            
            # 水素置換数の変化
            if mode == 'hydrogen':
                sc += sum( [ abs( self.rct.dic_hyd[x] - self.prd.dic_hyd[y] ) for x, y in mp ]  )
            
            # 結合次数の変化
            if mode == 'bond':
                for (x, y), (s, t) in combinations( mp, 2 ):
                    sc += abs( self.rct.dic_order[(x,s)] - self.prd.dic_order[(y,t)] )
            
            if mode == 'hydrogen + bond':
                sc += sum( [ abs( self.rct.dic_hyd[x] - self.prd.dic_hyd[y] ) for x, y in mp ]  )
                for (x, y), (s, t) in combinations( mp, 2 ):
                    sc += abs( self.rct.dic_order[(x,s)] - self.prd.dic_order[(y,t)] )
            
            scores.append( sc )
        
        # 最小スコアをリストに
        best_score = min( scores )
        best_maps = [ maps[i] for i, x in enumerate( scores ) if x == best_score ]
        
        #print( '# score calculation: mode =', mode )
        #print( '\t>>', len( best_maps ), 'best maps , score =', best_score, ', length =', len( best_maps[0] ) )
        #print( '\t>> time: ', str( round( time() - t, 6 ) ).rjust(4), 'sec' )
        
        return best_maps
    
    def calculate_mapping( self, mode='bond', optimizer='aki', timeout=10.0 ):
        
        t0 = time()
        
        #premap = self.find_unique_pair()
        premap = []
        modular = self.form_graph_product( mode='bond', neib=0, premap=premap )
        print('TEST1')
        maps = self.maximum_cliques( modular=modular, premap=premap, completion=True, optimizer='aki', timeout=timeout )
        print('TEST2')
        maps2 = self.permutation_complete( maps )
        
        print( '\t>>', len( maps2 ), 'maps were found..' )
        print( '>>>>> mapping time:', str( round( time()-t0, 6 ) ).rjust( 4 ) )
        
        return maps2
    
    def check_answer( self, maps ):
        
        answer = [ (x+1, x+1) for x, _ in enumerate( maps[0] ) ]
        
        for x in maps:
            gp = self.isomorphism( [ x, answer ], mode='skelton' )
            if len( gp ) == 1:
                return x
        
        return None

In [3]:
def is_correct_mapping( answer, test ):
    
    if test == '':
        return False
    
    RM_ans = ReactionMap( answer )
    RM_tst = ReactionMap( test )
    CGR_ans = RM_ans.RCT ^ RM_ans.PRD
    CGR_tst = RM_tst.RCT ^ RM_tst.PRD
    
    return CGR_ans == CGR_tst

In [6]:
with open( 'typical_smarts_2024_07_03.txt', 'r' ) as f:
    strs_ans = [ x.strip('\n') for x in f.readlines() ]

len( strs_ans )

241

In [7]:
with open( 'MarvinJS_result.txt', 'r' ) as f:
    strs_marvin = [ x.strip('\n') for x in f.readlines() ][8:]

len( strs_ans )

241

In [241]:
with open( 'ReactionMap_result.txt', 'r' ) as f:
    strs_rm = [ x.strip('\n') for x in f.readlines() ]

len( strs_rm )

241

In [55]:
with open( 'benchmark_ReactionMap_2.csv', 'w' ) as f:
    for i, x in enumerate( strs_ans ):
        print( i, '------------------------------' )
        f.write( str(i+1) +','+ str( is_correct_mapping( strs_ans[i], strs_rm[i] ) ) + '\n' )

0 ------------------------------
target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.011969 sec
target reaction: [CH3:2][CH2:1][SiH:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10](=[O:16])[CH2:11][CH2:12][C:13](=[O:15])[Cl:14]>>[CH3:2][CH2:1][Si:3]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH3:8][O:9][C:10](=[O:16])[CH2:11][CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.011272 sec
1 ------------------------------
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([OH:2])[C:14]=1[OH:15])[O:6][CH3:5]
# initialization: 
	>> time:  0.014039 sec
target reaction: [CH3:8][O:7][C:6]1=[CH:5][CH:13]=[CH:12][

# initialization: 
	>> time:  0.049036 sec
target reaction: [H:1][C:2]1([CH2:3][CH2:4][C:5]2([C:6](=[CH:7][CH2:8][C:9]3([C:10]2([CH2:11][CH2:12][C:13]4([C:14]3([CH2:15][CH2:16][C:17]4=[O:18])[H:19])[CH3:20])[H:21])[H:22])[CH2:23]1)[CH3:24])[OH:25].[CH3:26][C:27]([CH3:28])([CH3:29])[Si:30]([CH3:31])([CH3:32])[Cl:33].[CH:34]1=[CH:35][N:36]=[CH:37][NH:38]1>>[H:1][C:2]1([CH2:3][CH2:4][C:5]2([C:6](=[CH:7][CH2:8][C:9]3([C:10]2([CH2:11][CH2:12][C:13]4([C:14]3([CH2:15][CH2:16][C:17]4=[O:18])[H:19])[CH3:20])[H:21])[H:22])[CH2:23]1)[CH3:24])[O:25][Si:30]([CH3:31])([CH3:32])[C:27]([CH3:26])([CH3:28])[CH3:29].[CH:34]1=[CH:35][NH+:36]=[CH:37][NH:38]1.[Cl-:33]
# initialization: 
	>> time:  0.03205 sec
15 ------------------------------
target reaction: [CH3:1][C:2]([CH3:3])([CH3:4])[O:5][O:6][C:7]([CH3:8])([CH3:9])[CH3:10].[O:18]=[CH:11][C:12]1=[CH:13][CH:14]=[CH:15][CH:16]=[CH:17]1.[CH:27]2=[CH:26][CH:25]=[CH:24][CH:23]=[C:22]2[CH2:21][CH2:20][NH2:19]>>[CH3:1][C:2]([CH3:3])([CH3:4])[OH:5].[CH3:8][C:

# initialization: 
	>> time:  0.057801 sec
28 ------------------------------
target reaction: [CH2:19]1[CH2:18][CH:17]([CH2:22][CH2:21][CH2:20]1)[N-:23][CH:24]2[CH2:25][CH2:26][CH2:27][CH2:28][CH2:29]2.[CH3:14][CH:13]([CH3:15])[C:12]([O:11][CH3:10])=[O:16].[C:3]3(=[CH:4][CH:5]=[C:6]([Br:9])[CH:7]=[CH:8]3)[O:2][CH3:1]>>[CH2:19]1[CH2:18][CH:17]([CH2:22][CH2:21][CH2:20]1)[NH:23][CH:24]2[CH2:25][CH2:26][CH2:27][CH2:28][CH2:29]2.[CH3:14][C:13]([CH3:15])([C:6]3=[CH:5][CH:4]=[C:3]([CH:8]=[CH:7]3)[O:2][CH3:1])[C:12]([O:11][CH3:10])=[O:16].[Br-:9]
# initialization: 
	>> time:  0.017041 sec
target reaction: [CH3:14][CH:15]([CH3:16])[C:17](=[O:20])[O:18][CH3:19].[CH3:29][O:28][C:21]1=[CH:22][CH:23]=[C:24]([CH:26]=[CH:27]1)[Br:25].[CH2:6]1[CH2:1][CH2:2][CH:3]([CH2:4][CH2:5]1)[N-:7][CH:8]2[CH2:9][CH2:10][CH2:11][CH2:12][CH2:13]2>>[CH3:14][C:15]([CH3:16])([C:24]1=[CH:26][CH:27]=[C:21]([CH:22]=[CH:23]1)[O:28][CH3:29])[C:17](=[O:20])[O:18][CH3:19].[CH2:6]1[CH2:1][CH2:2][CH:3]([CH2:4][CH2:5]1)[NH:7][CH

# initialization: 
	>> time:  0.02809 sec
40 ------------------------------
target reaction: [F:11][C:10]([F:12])([F:13])[C:7]=1[CH:8]=[N:9][C:4]([CH2:3][C:2](=[O:1])[C:14]=2[CH:20]=[CH:19][CH:18]=[C:16]([Br:17])[CH:15]=2)=[CH:5][CH:6]=1.[OH:22][NH2:21]>>[F:11][C:10]([F:12])([F:13])[C:7]=1[CH:8]=[N:9][C:4](=[CH:5][CH:6]=1)[CH2:3][C:2]([C:14]=2[CH:20]=[CH:19][CH:18]=[C:16]([Br:17])[CH:15]=2)=[N:21][OH:22].[OH2:1]
# initialization: 
	>> time:  0.010056 sec
target reaction: [CH:14]1=[CH:13][C:12](=[CH:18][C:16](=[CH:15]1)[Br:17])[C:10](=[O:11])[CH2:9][C:8]2=[CH:19][CH:20]=[C:5]([CH:6]=[N:7]2)[C:2]([F:1])([F:3])[F:4].[NH2:22][OH:21]>>[CH:14]1=[CH:13][C:12](=[CH:18][C:16](=[CH:15]1)[Br:17])[C:10](=[N:22][OH:21])[CH2:9][C:8]2=[CH:19][CH:20]=[C:5]([CH:6]=[N:7]2)[C:2]([F:1])([F:3])[F:4].[OH2:11]
# initialization: 
	>> time:  0.009974 sec
41 ------------------------------
target reaction: [F:1][C:2]([F:3])([F:4])[C:5]1=[CH:6][CH:7]=[C:8]([N:10]=[CH:11]1)[Cl:9].[CH:20]=2[CH:19]=[C:17]([Br:18])[C

# initialization: 
	>> time:  0.009042 sec
target reaction: [CH3:13][C:12]1=[CH:11][C:10](=[C:9]([CH:16]=[C:14]1[CH3:15])[CH3:18])[CH3:17].[N:2](=[O:1])(=[O:3])[OH:4].[N:6](=[O:5])(=[O:7])[OH:8]>>[CH3:18][C:9]1=[C:10]([C:11](=[C:12]([C:14](=[C:16]1[N+:2](=[O:3])[O-:1])[CH3:15])[CH3:13])[N+:6](=[O:7])[O-:5])[CH3:17].[OH2:4].[OH2:8]
# initialization: 
	>> time:  0.013029 sec
59 ------------------------------
target reaction: [CH3:5][C:6]([CH3:7])([CH3:39])[O:8][C:9]([N:10]1[CH2:11][CH2:12][N:13]([CH2:36][CH2:37]1)[C:14](=[O:35])[C:15]2=[CH:16][CH:17]=[CH:18][C:19]([N:20](=[O:21])=[O:22])=[C:23]2[CH:24]=[CH:25][C:26]=3[CH:31]=[CH:30][C:29]=4[O:32][CH2:33][O:34][C:28]=4[CH:27]=3)=[O:38].[O+:2]#[C-:1].[O+:4]#[C-:3]>>[O:2]=[C:1]=[O:21].[O:4]=[C:3]=[O:22].[CH3:5][C:6]([CH3:7])([CH3:39])[O:8][C:9]([N:10]1[CH2:11][CH2:12][N:13]([CH2:36][CH2:37]1)[C:14]([C:15]=2[C:23]=3[CH:24]=[C:25]([C:26]4=[CH:27][C:28]=5[O:34][CH2:33][O:32][C:29]=5[CH:30]=[CH:31]4)[NH:20][C:19]=3[CH:18]=[CH:17][CH:16]=2)=[O:3

...

# initialization: 
	>> time:  0.012037 sec
147 ------------------------------
target reaction: [CH2:13]([OH:12])[CH3:14].[CH2:16]([OH:17])[CH3:15].[CH2:6]1[CH:5]2[CH:10]([CH2:9][CH:8]=[CH:7]1)[C:2](=[O:1])[O:3][C:4]2=[O:11]>>[CH:7]1=[CH:8][CH2:9][CH:10]([CH:5]([C:4](=[O:11])[O:17][CH2:16][CH3:15])[CH2:6]1)[C:2](=[O:1])[O:12][CH2:13][CH3:14].[OH2:3]
# initialization: 
	>> time:  0.00805 sec
target reaction: [CH3:3][CH2:1][OH:2].[CH3:6][CH2:4][OH:5].[CH2:7]1[CH:12]=[CH:11][CH2:10][CH:9]2[CH:8]1[C:16](=[O:17])[O:15][C:13]2=[O:14]>>[CH3:6][CH2:4][O:15][C:13](=[O:14])[CH:9]1[CH2:10][CH:11]=[CH:12][CH2:7][CH:8]1[C:16](=[O:17])[O:2][CH2:1][CH3:3].[OH2:5]
# initialization: 
	>> time:  0.007978 sec
148 ------------------------------
target reaction: [CH:14]1=[CH:15][CH2:16][CH:6]([CH:7]([C:8](=[O:12])[O:9][CH2:10][CH3:11])[CH2:13]1)[C:2](=[O:1])[O:3][CH2:4][CH3:5]>>[O:3]([C:2]([CH:6]1[CH:7]([C:8]([O:9][CH2:10][CH3:11])=[O:12])[CH2:13][CH2:14][CH2:15][CH2:16]1)=[O:1])[CH2:4][CH3:5]
# initializat

# initialization: 
	>> time:  0.00399 sec
target reaction: [CH3:1][CH:2]([CH3:3])[C:4](=[O:6])[CH3:5].[Br:7][Br:8]>>[CH3:1][CH:2]([CH3:3])[C:4](=[O:6])[CH2:5][Br:7].[BrH:8]
# initialization: 
	>> time:  0.004986 sec
171 ------------------------------
172 ------------------------------
target reaction: [C:8]1=2[NH:9][CH:10]=[CH:11][C:3]1=[CH:4][CH:5]=[CH:6][CH:7]=2.[CH3:1][I:2]>>[CH:7]=1[CH:6]=[CH:5][CH:4]=[C:3]2[CH:11]=[CH:10][N:9]([CH3:1])[C:8]=12.[IH:2]
# initialization: 
	>> time:  0.004988 sec
target reaction: [CH3:10][I:11].[CH:7]1=[CH:8][CH:9]=[C:1]2[C:5](=[CH:6]1)[CH:4]=[CH:3][NH:2]2>>[CH3:10][N:2]1[CH:3]=[CH:4][C:5]2=[CH:6][CH:7]=[CH:8][CH:9]=[C:1]21.[IH:11]
# initialization: 
	>> time:  0.005157 sec
173 ------------------------------
target reaction: [OH:12][C:11]([CH2:10][C:9]#[N:8])=[O:13].[O:1]=[CH:2][C:3]=1[O:7][CH:6]=[CH:5][CH:4]=1>>[O:12]=[C:11]=[O:13].[OH2:1].[N:8]#[C:9][CH:10]=[CH:2][C:3]=1[O:7][CH:6]=[CH:5][CH:4]=1
# initialization: 
	>> time:  0.004986 sec
target rea

# initialization: 
	>> time:  0.007979 sec
192 ------------------------------
target reaction: [O:3]([Li:4])[CH:2]=[CH2:1].[CH:8]1=[C:9]([CH:11]=[CH:12][C:6]([Cl:5])=[CH:7]1)[CH3:10]>>[C:6]=12[C:7](=[CH:8][C:9](=[CH:11][CH:12]=1)[CH3:10])[CH:2]([O:3][Li:4])[CH2:1]2.[ClH:5]
# initialization: 
	>> time:  0.006056 sec
target reaction: [Li:2][O:1][CH:3]=[CH2:4].[CH3:12][C:6]1=[CH:5][CH:11]=[C:9]([CH:8]=[CH:7]1)[Cl:10]>>[Li:2][O:1][CH:3]1[CH2:4][C:9]2=[CH:8][CH:7]=[C:6]([CH:5]=[C:11]12)[CH3:12].[ClH:10]
# initialization: 
	>> time:  0.006983 sec
193 ------------------------------
target reaction: [C:10]=12[C:4](=[CH:5][C:6](=[CH:8][CH:9]=1)[CH3:7])[CH:3]([O:2][Li:1])[CH2:11]2>>[C:4]\1([CH:5]=[C:6]([CH:8]=[CH:9][C:10]/1=[CH2:11])[CH3:7])=[CH:3]/[O:2][Li:1]
# initialization: 
	>> time:  0.00711 sec
target reaction: [Li:10][O:9][CH:8]1[CH2:11][C:1]2=[CH:6][CH:5]=[C:4]([CH:3]=[C:2]12)[CH3:7]>>[Li:10][O:9][CH:8]=[C:2]1[CH:3]=[C:4]([CH:5]=[CH:6][C:1]1=[CH2:11])[CH3:7]
# initialization: 
	>> time:

# initialization: 
	>> time:  0.005983 sec
target reaction: [CH:3]1=[CH:2][CH:1]=[C:6]([CH:5]=[CH:4]1)[N:7]=[N:8][C:9]2=[CH:10][CH:11]=[CH:12][CH:13]=[CH:14]2>>[CH:2]1=[CH:1][C:6](=[CH:5][CH:4]=[C:3]1[C:12]2=[CH:11][CH:10]=[C:9]([CH:14]=[CH:13]2)[NH2:8])[NH2:7]
# initialization: 
	>> time:  0.006056 sec
217 ------------------------------
218 ------------------------------
target reaction: [CH2:1]=[C:2]1[C:7](=[CH2:8])[CH:6]=[CH:5][CH:4]=[CH:3]1>>[CH2:1]1[CH2:8][C:7]2=[C:2]1[CH:3]=[CH:4][CH:5]=[CH:6]2
# initialization: 
	>> time:  0.003861 sec
target reaction: [CH2:1]=[C:2]1[CH:8]=[CH:7][CH:6]=[CH:5][C:3]1=[CH2:4]>>[CH2:1]1[CH2:4][C:3]2=[C:2]1[CH:8]=[CH:7][CH:6]=[CH:5]2
# initialization: 
	>> time:  0.003989 sec
219 ------------------------------
target reaction: [CH3:11][C:12]([CH3:13])([CH3:14])[O-:15].[CH3:1][CH:2]=[O:3].[CH3:4][CH2:5][O:6][C:7]([CH2:9][Cl:10])=[O:8]>>[CH3:11][C:12]([CH3:13])([CH3:14])[OH:15].[O:8]=[C:7]([CH:9]1[O:3][CH:2]1[CH3:1])[O:6][CH2:5][CH3:4].[Cl-:10]
# initi

# initialization: 
	>> time:  0.108159 sec
target reaction: [B:9]1([CH:8]2[CH2:1][CH2:2][CH2:3][CH:4]1[CH2:5][CH2:6][CH2:7]2)[CH2:10][CH:11]=[C:12]([CH3:13])[CH3:14].[CH3:15][C:16]1([CH:25]2[CH2:26][CH2:27][C:28]([CH:30]([CH:24]2[C:23]3=[CH:22][NH:21][C:20]4=[CH:35][CH:36]=[CH:37][C:18]1=[C:19]43)[N+:31]#[C-:32])([CH3:29])[CH:33]=[CH2:34])[CH3:17]>>[BH:9]1[CH:8]2[CH2:1][CH2:2][CH2:3][CH:4]1[CH2:5][CH2:6][CH2:7]2.[CH3:15][C:16]1([CH:25]2[CH2:26][CH2:27][C:28]([CH:30]([CH:24]2[C:23]3=[C:22]([NH:21][C:20]4=[CH:35][CH:36]=[CH:37][C:18]1=[C:19]43)[C:12]([CH3:13])([CH3:14])[CH:11]=[CH2:10])[N+:31]#[C-:32])([CH3:29])[CH:33]=[CH2:34])[CH3:17]
# initialization: 
	>> time:  0.029541 sec
233 ------------------------------
target reaction: [O:11]=[C:10]([O:9][CH2:8][CH3:7])[C:12]([C:14](=[O:15])[O:16][CH2:17][CH3:18])=[O:13].[CH3:1][O:2]/[CH:3]=[CH:4]/[CH:5]=[CH2:6]>>[CH2:8]([CH3:7])[O:9][C:10](=[O:11])[C:12]1([C:14]([O:16][CH2:17][CH3:18])=[O:15])[O:13][CH:3]([O:2][CH3:1])[CH:4]=[CH:5][CH2:6]1
# 

In [142]:
with open( 'benchmark_marvin.csv', 'w' ) as f:
    for i, x in enumerate( strs_ans ):
        f.write( str(i+1) +','+ str( is_correct_mapping( strs_ans[i], strs_marvin[i] ) ) + '\n' )

target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.00698 sec
target reaction: [CH3:2][CH2:1][SiH:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10](=[O:16])[CH2:11][CH2:12][C:13]([Cl:14])=[O:15]>>[CH3:2][CH2:1][Si:3]([Cl:14])([CH2:6][CH3:7])[CH2:4][CH3:5].[CH3:8][O:9][C:10](=[O:16])[CH2:11][CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.009053 sec
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([OH:2])[C:14]=1[OH:15])[O:6][CH3:5]
# initialization: 
	>> time:  0.005985 sec
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH3:8][O:7][C:6]1=[CH:5][CH:13]=[CH:12][C:11]([CH:14]=[O:15])=[C:9]1[OH:10]>>[OH2

# initialization: 
	>> time:  0.015012 sec
target reaction: [C:10]1(=[O:11])[CH2:12][CH2:13][CH2:14][C:15](=[O:16])[C:6]1([CH2:5][CH2:4][C:2](=[O:3])[CH3:1])[CH2:7][CH:8]=[CH2:9]>>[CH2:4]1[CH2:5][C@:6]2([C:10]([CH2:12][CH2:13][CH2:14][C:15]2=[O:16])=[CH:1][C:2]1=[O:3])[CH2:7][CH:8]=[CH2:9].[OH2:11]
# initialization: 
	>> time:  0.00805 sec
target reaction: [CH3:13][C:11](=[O:12])[CH2:10][CH2:9][C:8]1([CH2:14][CH:15]=[CH2:16])[C:1](=[O:2])[CH2:3][CH2:4][CH2:5][C:6]1=[O:7]>>[CH2:16]=[CH:15][CH2:14][C@:8]12[CH2:9][CH2:10][C:11](=[O:12])[CH:13]=[C:6]1[CH2:5][CH2:4][CH2:3][C:1]2=[O:2].[OH2:7]
# initialization: 
	>> time:  0.009974 sec
target reaction: [CH3:1][CH:2]([CH3:3])[N-:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19]([C@H:8]2[N:12]1[CH2:11][CH2:10][CH2:9]2)=[O:20].[CH:23](=[CH2:24])[CH2:22][Br:21]>>[CH3:1][CH:2]([CH3:3])[NH:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19](=[O:20])[C@@:8]2([CH2:9][CH2:10][CH2:11][N:12]12)[CH2:

# initialization: 
	>> time:  0.01685 sec
target reaction: [CH2:6]1[CH2:1][CH2:2][CH:3]([CH2:4][CH2:5]1)[N-:7][CH:8]1[CH2:9][CH2:10][CH2:11][CH2:12][CH2:13]1.[CH3:14][N:15]([CH3:16])[C:17]1=[CH:23][CH:22]=[C:20]([Br:21])[CH:19]=[CH:18]1.[CH3:29][O:28][C:27](=[O:30])[CH:25]([CH3:24])[CH3:26]>>[CH2:6]1[CH2:1][CH2:2][CH:3]([CH2:4][CH2:5]1)[NH:7][CH:8]1[CH2:9][CH2:10][CH2:11][CH2:12][CH2:13]1.[CH3:29][O:28][C:27](=[O:30])[C:25]([CH3:24])([CH3:26])[C:20]1=[CH:22][CH:23]=[C:17]([CH:18]=[CH:19]1)[N:15]([CH3:14])[CH3:16].[Br-:21]
# initialization: 
	>> time:  0.017073 sec
target reaction: [CH:6]=1[CH:5]=[CH:4][CH:3]=[C:2]([CH:1]=[O:8])[CH:7]=1.[C:11]2(=[C:10]([CH:15]=[CH:14][CH:13]=[CH:12]2)[CH3:9])[NH2:16]>>[NH:16]([CH2:1][C:2]=1[CH:3]=[CH:4][CH:5]=[CH:6][CH:7]=1)[C:11]=2[CH:12]=[CH:13][CH:14]=[CH:15][C:10]=2[CH3:9].[OH2:8]
# initialization: 
	>> time:  0.005984 sec
target reaction: [O:7]=[CH:6][C:5]1=[CH:4][CH:3]=[CH:2][CH:1]=[CH:8]1.[CH3:15][C:10]1=[C:9]([NH2:16])[CH:14]=[CH:13][CH:12]=[CH:

# initialization: 
	>> time:  0.012966 sec
target reaction: [CH:8]=1[CH:9]=[CH:10][C:5](=[C:6]([CH:7]=1)[NH2:11])[NH2:4].[N:2]([OH:3])=[O:1]>>[OH2:1].[OH2:3].[CH:9]=1[CH:10]=[C:5]2[N:4]=[N:2][NH:11][C:6]2=[CH:7][CH:8]=1
# initialization: 
	>> time:  0.004987 sec
target reaction: [NH2:7][C:5]1=[C:4]([NH2:8])[CH:3]=[CH:2][CH:1]=[CH:6]1.[OH:10][N:9]=[O:11]>>[OH2:10].[OH2:11].[NH:7]1[N:9]=[N:8][C:4]2=[CH:3][CH:2]=[CH:1][CH:6]=[C:5]12
# initialization: 
	>> time:  0.004987 sec
target reaction: [C-:1]#[O+:2].[C-:3]#[O+:4].[CH2:10]1[CH2:9][N:8]([CH2:34][CH2:33][N:11]1[C:12]([C:13]=2[C:21](=[C:17]([CH:16]=[CH:15][CH:14]=2)[N:18](=[O:19])=[O:20])[CH:22]=[CH:23][C:24]3=[CH:25][C:26](=[CH:28][C:29](=[CH:31]3)[F:30])[Cl:27])=[O:32])[S:6](=[O:35])(=[O:7])[CH3:5]>>[O:2]=[C:1]=[O:19].[O:4]=[C:3]=[O:20].[CH2:10]1[CH2:9][N:8]([CH2:34][CH2:33][N:11]1[C:12]([C:13]=2[C:21]3=[C:17]([CH:16]=[CH:15][CH:14]=2)[NH:18][C:23](=[CH:22]3)[C:24]=4[CH:31]=[C:29]([F:30])[CH:28]=[C:26]([Cl:27])[CH:25]=4)=[O:32])[S:6](

# initialization: 
	>> time:  0.015624 sec
target reaction: [CH2:3]([CH2:2][CH3:1])[CH2:4][CH:5]([CH2:6][CH3:7])[O:8][S:9][C:10]=1[CH:11]=[CH:12][CH:13]=[CH:14][CH:15]=1>>[CH:5]([CH2:4][CH2:3][CH:2]([S:9][C:10]=1[CH:11]=[CH:12][CH:13]=[CH:14][CH:15]=1)[CH3:1])([OH:8])[CH2:6][CH3:7]
# initialization: 
	>> time:  0.009539 sec
target reaction: [CH3:3][CH2:2][CH2:1][CH2:4][CH:5]([CH2:6][CH3:7])[O:8][S:9][C:10]1=[CH:15][CH:14]=[CH:13][CH:12]=[CH:11]1>>[CH3:7][CH2:6][CH:5]([OH:8])[CH2:4][CH2:1][CH:2]([CH3:3])[S:9][C:10]1=[CH:11][CH:12]=[CH:13][CH:14]=[CH:15]1
# initialization: 
	>> time:  0.005983 sec
target reaction: [CH2:25]1[CH2:26][CH:27]([CH2:38][CH2:39][N:24]1[C:23]([C:22]2=[CH:21][CH:20]=[CH:19][C:15]([N:16](=[O:17])=[O:18])=[C:14]2[CH:13]=[CH:12][C:9]3=[CH:8][CH:7]=[C:6]([F:5])[CH:11]=[CH:10]3)=[O:40])[N:28]4[C:32]5=[C:31]([CH:36]=[CH:35][CH:34]=[CH:33]5)[NH:30][C:29]4=[O:37].[O+:2]#[C-:1].[O+:4]#[C-:3]>>[O:2]=[C:1]=[O:17].[O:4]=[C:3]=[O:18].[CH2:25]1[CH2:26][CH:27]([CH2:38][CH2:39][

...

# initialization: 
	>> time:  0.015622 sec
target reaction: [CH3:21][O:20][CH:19]([CH3:22])[N:8]1[CH:7]([CH:12]([O:11][C:9]1=[O:10])[C:13]1=[CH:18][CH:17]=[CH:16][CH:15]=[CH:14]1)[C:6]1=[CH:1][CH:2]=[CH:3][CH:4]=[CH:5]1>>[CH2:22]=[CH:19][N:8]1[CH:7]([CH:12]([O:11][C:9]1=[O:10])[C:13]1=[CH:18][CH:17]=[CH:16][CH:15]=[CH:14]1)[C:6]1=[CH:5][CH:4]=[CH:3][CH:2]=[CH:1]1.[CH3:21][OH:20]
# initialization: 
	>> time:  0.014205 sec
target reaction: [I:8][I:9].[CH2:5]1[CH:4]=[CH:3][C:2](=[O:1])[CH2:7][CH2:6]1>>[O:1]=[C:2]1[C:3]([I:9])=[CH:4][CH2:5][CH2:6][CH2:7]1.[IH:8]
# initialization: 
	>> time:  0.003989 sec
target reaction: [I:1][I:2].[O:7]=[C:6]1[CH2:8][CH2:9][CH2:3][CH:4]=[CH:5]1>>[I:1][C:5]1=[CH:4][CH2:3][CH2:9][CH2:8][C:6]1=[O:7].[IH:2]
# initialization: 
	>> time:  0.003071 sec
target reaction: [CH3:1][CH:2]=[O:3].[CH2:4]=[CH:5][C:6](=[O:9])[O:7][CH3:8]>>[O:9]=[C:6]([C:5](=[CH2:4])[CH:2]([OH:3])[CH3:1])[O:7][CH3:8]
# initialization: 
	>> time:  0.003989 sec
target reaction: [CH3:1][CH:2]

# initialization: 
	>> time:  0.015624 sec
target reaction: [CH3:9][C:10](=[O:15])[O:11][C:12]([CH3:13])=[O:14].[CH2:5]1[CH2:4][CH2:3][CH:2]([C:7](=[O:8])[CH2:6]1)[CH3:1]>>[CH2:3]1[C:2]([CH3:1])=[C:7]([CH2:6][CH2:5][CH2:4]1)[O:8][C:12](=[O:14])[CH3:13].[OH:11][C:10]([CH3:9])=[O:15]
# initialization: 
	>> time:  0.006982 sec
target reaction: [CH3:1][C:2](=[O:3])[O:4][C:5]([CH3:6])=[O:7].[CH3:15][CH:11]1[CH2:10][CH2:9][CH2:8][CH2:14][C:12]1=[O:13]>>[CH3:1][C:2](=[O:3])[O:13][C:12]1=[C:11]([CH3:15])[CH2:10][CH2:9][CH2:8][CH2:14]1.[CH3:6][C:5]([OH:4])=[O:7]
# initialization: 
	>> time:  0.006981 sec
target reaction: [F:19][C:18]([F:20])([F:21])[C:17]=1[CH:16]=[C:15]([CH:24]=[CH:23][CH:22]=1)[N:13]([CH3:12])[CH3:14].[Br:4][C:3]2=[CH:5][C:6]([CH:9]=[C:10]([Br:11])[C:2]2=[O:1])([Br:8])[Br:7]>>[F:19][C:18]([F:20])([F:21])[C:17]1=[C:22]([CH:23]=[CH:24][C:15](=[CH:16]1)[N:13]([CH3:12])[CH3:14])[Br:8].[C:6]2(=[CH:9][C:10](=[C:2]([C:3](=[CH:5]2)[Br:4])[OH:1])[Br:11])[Br:7]
# initialization: 
	>> t

# initialization: 
	>> time:  0.024124 sec
target reaction: [CH3:16][O:15][C:13](=[O:14])[C@H:8]([CH2:7][C:6]1=[CH:1][CH:2]=[CH:3][CH:4]=[CH:5]1)[NH:9][CH2:10][CH:11]=[CH2:12].[Br:20][CH2:19][C:17]#[CH:18]>>[CH3:16][O:15][C:13](=[O:14])[C@H:8]([CH2:7][C:6]1=[CH:1][CH:2]=[CH:3][CH:4]=[CH:5]1)[N:9]([CH2:10][CH:11]=[CH2:12])[CH2:19][C:17]#[CH:18].[BrH:20]
# initialization: 
	>> time:  0.011967 sec
target reaction: [C:2]=1([C:7]([NH2:8])=[CH:6][CH:5]=[CH:4][CH:3]=1)[NH2:1].[O:11]([C:10]([C:13]([CH2:14][CH3:15])=[O:16])=[O:9])[CH3:12]>>[CH:5]=1[CH:6]=[C:7]2[N:8]=[C:13]([C:10](=[N:1][C:2]2=[CH:3][CH:4]=1)[OH:9])[CH2:14][CH3:15].[OH2:16].[CH3:12][OH:11]
# initialization: 
	>> time:  0.006073 sec
target reaction: [NH2:3][C:2]1=[CH:4][CH:5]=[CH:6][CH:7]=[C:1]1[NH2:8].[CH3:13][CH2:12][C:11](=[O:14])[C:10](=[O:15])[O:9][CH3:16]>>[CH3:10][CH2:11][C:12]1=[N:3][C:2]2=[CH:4][CH:5]=[CH:6][CH:7]=[C:1]2[N:8]=[C:13]1[OH:14].[OH2:15].[CH3:16][OH:9]
# initialization: 
	>> time:  0.006981 sec
target reactio

# initialization: 
	>> time:  0.018257 sec
target reaction: [CH3:11][C:10]1([CH3:16])[O:12][C@H:13]([CH2:14][OH:15])[C@H:8](/[CH:7]=[CH:6]/[CH2:5][CH2:4][C:3](=[O:17])[O:2][CH3:1])[O:9]1>>[CH3:11][C:10]1([CH3:16])[O:12][C@@H:13]([C@@H:8]([O:9]1)/[CH:7]=[CH:6]/[CH2:5][CH2:4][C:3](=[O:17])[O:2][CH3:1])[CH:14]=[O:15]
# initialization: 
	>> time:  0.009973 sec
target reaction: [CH3:16][O:15][C:13](=[O:14])[CH2:12][CH2:11]\[CH:10]=[CH:9]\[C@@H:8]1[O:17][C:2]([CH3:1])([CH3:3])[O:4][C@@H:5]1[CH2:6][OH:7]>>[CH3:16][O:15][C:13](=[O:14])[CH2:12][CH2:11]\[CH:10]=[CH:9]\[C@@H:8]1[O:17][C:2]([CH3:1])([CH3:3])[O:4][C@@H:5]1[CH:6]=[O:7]
# initialization: 
	>> time:  0.011068 sec
target reaction: [CH3:12][C:11]1([CH3:17])[O:13][C@@H:14]([C@@H:9]([O:10]1)/[CH:8]=[CH:7]/[CH2:6][CH2:5][C:4](=[O:18])[O:3][CH3:2])[CH:15]=[O:16].[OH2:1]>>[CH3:12][C:11]1([CH3:17])[O:10][C@@H:9](/[CH:8]=[CH:7]/[CH2:6][CH2:5][C:4](=[O:18])[O:3][CH3:2])[C@H:14]([O:13]1)[C:15]([OH:1])=[O:16]
# initialization: 
	>> time:  0.01301

# initialization: 
	>> time:  0.043434 sec
target reaction: [CH3:13][C:12]([CH3:14])=[CH:11][CH2:10][B:9]1[CH:8]2[CH2:1][CH2:2][CH2:3][CH:4]1[CH2:5][CH2:6][CH2:7]2.[CH3:29][C:28]1([CH2:27][CH2:26][CH:25]2[CH:24]([CH:30]1[N+:31]#[C-:32])[C:23]1=[CH:22][NH:21][C:20]3=[CH:35][CH:36]=[CH:37][C:18](=[C:19]13)[C:16]2([CH3:15])[CH3:17])[CH:33]=[CH2:34]>>[BH:9]1[CH:8]2[CH2:1][CH2:2][CH2:3][CH:4]1[CH2:5][CH2:6][CH2:7]2.[CH3:13][C:12]([CH3:14])([CH:11]=[CH2:10])[C:22]1=[C:23]2[CH:24]3[CH:25]([CH2:26][CH2:27][C:28]([CH3:29])([CH:33]=[CH2:34])[CH:30]3[N+:31]#[C-:32])[C:16]([CH3:15])([CH3:17])[C:18]3=[C:19]2[C:20]([NH:21]1)=[CH:35][CH:36]=[CH:37]3
# initialization: 
	>> time:  0.034132 sec
target reaction: [O:11]=[C:10]([O:9][CH2:8][CH3:7])[C:12]([C:14](=[O:15])[O:16][CH2:17][CH3:18])=[O:13].[CH3:1][O:2]/[CH:3]=[CH:4]/[CH:5]=[CH2:6]>>[CH2:8]([CH3:7])[O:9][C:10](=[O:11])[C:12]1([C:14]([O:16][CH2:17][CH3:18])=[O:15])[O:13][CH:3]([O:2][CH3:1])[CH:4]=[CH:5][CH2:6]1
# initialization: 
	>> time:  0.00797

In [265]:
i += 1
RM = ReactionMap( strs_rm[i] )
print( i )
RM.RXN

target reaction: 


ValueError: not enough values to unpack (expected 2, got 1)

In [224]:
RM = ReactionMap( strs_ans[i] )
RM.RXN

target reaction: [CH2:4]1[CH2:5][CH:3]1[CH2:2][NH2:1].[N:7]([OH:6])=[O:8]>>[CH2:2]1[CH2:5][CH:4]=[CH:3]1.[OH2:6].[OH2:8].[N:1]#[N:7]
# initialization: 
	>> time:  0.004986 sec


In [242]:
i = 239 - 1
RM = ReactionMap( strs_rm[i] )
cgr = RM.RCT ^ RM.PRD
cgr.clean2d()
cgr

target reaction: [CH2:11]=[CH:10][CH:9]=[O:8].[CH:2]1=[CH:1][CH:6]=[C:5]([CH:4]=[CH:3]1)[NH2:7]>>[CH:2]1=[CH:3][C:4]2=[C:5]([CH:6]=[CH:1]1)[N:7]=[CH:9][CH:10]=[CH:11]2.[OH2:8]
# initialization: 
	>> time:  0.006981 sec


In [243]:
RM = ReactionMap( strs_ans[i] )
cgr = RM.RCT ^ RM.PRD
cgr.clean2d()
cgr

target reaction: [CH:6]1=[CH:5][CH:4]=[CH:3][C:2](=[CH:7]1)[NH2:1].[O:11]=[CH:10][CH:9]=[CH2:8]>>[C:3]12=[C:2]([CH:7]=[CH:6][CH:5]=[CH:4]1)[N:1]=[CH:8][CH:9]=[CH:10]2.[OH2:11]
# initialization: 
	>> time:  0.005984 sec


In [267]:
m = smiles('[Br:1][Mg:2][CH:3]=[CH2:4].[CH3:5][C:6]1([CH3:15])[O:7][C@@H:8]2[CH2:9][O:10][CH:11]([OH:14])[C@@H:12]2[O:13]1>>[MgH:2][Br:1].[CH3:15][C:6]1([CH3:5])[O:7][C@H:8]([CH2:9][OH:10])[C@@H:12]([O:13]1)[C@@H:11]([OH:14])[CH:3]=[CH2:4]')
m.clean2d()
m

In [268]:
m.clean2d()
m

In [66]:
for x in gps:
    x.sort()

for x in gps2:
    x.sort()

for x in gps3:
    x.sort()

for x in gps3:
    for y in gps2:
        if x == y:
            print( y )
    for z in gps:
        if x == z:
            print( z )

In [37]:
RM.RXN

In [41]:
RM = ReactionMap('[CH3:8][O:7][C:6]1=[CH:5][CH:13]=[CH:12][C:11](=[C:9]1[OH:10])[CH:14]=[O:15].[OH:1][OH:2].[OH:3][OH:4]>>[CH3:8][O:7][C:6]1=[C:9]([C:11](=[CH:12][CH:13]=[CH:5]1)[OH:4])[OH:10].[C:14](=[O:15])=[O:3].[OH2:1].[OH2:2]')
RM.RXN

target reaction: [CH3:8][O:7][C:6]1=[CH:5][CH:13]=[CH:12][C:11](=[C:9]1[OH:10])[CH:14]=[O:15].[OH:1][OH:2].[OH:3][OH:4]>>[CH3:8][O:7][C:6]1=[C:9]([C:11](=[CH:12][CH:13]=[CH:5]1)[OH:4])[OH:10].[C:14](=[O:15])=[O:3].[OH2:1].[OH2:2]
# initialization: 
	>> time:  0.009973 sec


In [3]:
def make_CGR( RM, mp ):
    R = deepcopy( RM.RCT )
    R.remap( dict( mp ) )
    CGR = R ^ RM.PRD
    CGR.clean2d()
    return CGR

In [4]:
with open( 'dataset_typical_241_reactions.txt', 'r' ) as f:
    strns = [ x.strip() for x in f.readlines() ]

In [5]:
def show_rdkit( info ):
    from rdkit import Chem
    from rdkit.Chem import Draw
    from rdkit.Chem.Draw import rdMolDraw2D
    
    strn, mp, clev, form = info
    
    rct, prd = strn.split('>>')
    r_mol = Chem.MolFromSmiles( rct )
    r_mol.UpdatePropertyCache( strict = False )
    p_mol = Chem.MolFromSmiles( prd )
    p_mol.UpdatePropertyCache( strict = False )
    
    rdkit_r_bonds = dict()
    for x in r_mol.GetBonds():
        s = x.GetBeginAtom().GetAtomMapNum()
        t = x.GetEndAtom().GetAtomMapNum()
        rdkit_r_bonds[(s, t)] = x.GetIdx()
        rdkit_r_bonds[(t, s)] = x.GetIdx()
    
    rdkit_p_bonds = dict()
    for x in p_mol.GetBonds():
        s = x.GetBeginAtom().GetAtomMapNum()
        t = x.GetEndAtom().GetAtomMapNum()
        rdkit_p_bonds[(s, t)] = x.GetIdx()
        rdkit_p_bonds[(t, s)] = x.GetIdx()
    
    clev_bonds = [ rdkit_r_bonds[x] for x in clev if x in rdkit_r_bonds.keys() ]
    form_bonds = [ rdkit_p_bonds[x] for x in form if x in rdkit_p_bonds.keys() ]
    
    idx_conv = dict( [ (y, x) for x, y in mp ] )
    # print( 'conv clev', clev )
    # print( 'conv form', [ ( idx_conv[x], idx_conv[y] ) for x, y in form ] )
    # print( idx_conv )
    
    for x in p_mol.GetAtoms():
        origin = x.GetAtomMapNum()
        x.SetAtomMapNum( idx_conv[origin] )
        #print( origin, '->', idx_conv[origin] )
    
    return Draw.MolsToGridImage( mols=[r_mol, p_mol], subImgSize=(300, 300), highlightBondLists=[clev_bonds, form_bonds] )

In [6]:
def visiualization( rxn_str ):
    RM = ReactionMap( rxn_str )
    maps = RM.calculate_mapping( mode='bond', optimizer='aki', timeout=1.0 )
    
    maps_info = []
    if len( maps[0] ) == len( RM.rct.atoms ):
        res = RM.check_answer( maps )
        
        #if res != None:
        #    print( '### correct', term, '\n', res )
        #else:
        #    print( '### wrong', term, '\n', maps[0] )
        
        for mp in maps:
            clev, form = RM.change_bonds( mp )
            maps_info.append((rxn_str, mp, clev, form))
            print( mp )
            print( 'cleavage ', clev )
            print( 'formation', form )
    
    else:
        print( '### imcomplete', term, '\n', maps[0] )
    print( '' )
    
    return maps_info

In [7]:
def only_visiualization( maps, RM, rxn_str ):
    
    maps_info = []
    for mp in maps:
        print( mp )
        clev, form = RM.change_bonds( mp )
        maps_info.append((rxn_str, mp, clev, form))
    
    return maps_info

In [45]:
def connected_comp( idxs, adj ):
    
    adj_dic = dict()
    for i, j in adj:
        if i in adj_dic.keys():
            adj_dic[i].add( j )
        else:
            adj_dic[i] = { j }
        if j in adj_dic.keys():
            adj_dic[j].add( i )
        else:
            adj_dic[j] = { i }        
    
    def next_adj( adj_dic, pivot ):
        nexts = set()
        for p in pivot:
            if not p in adj_dic.keys():
                nexts |= {p}
            else:
                nexts |= adj_dic[p]
        
        return nexts
    
    visited = set()
    cand = set()
    covered = set()
    connected_comp = []
    
    while covered != set( idxs ):
        residue = set( idxs ) - covered 
        cand = { residue.pop() }
        visited = cand
        
        while cand != set():
            visited |= cand
            cand = next_adj( adj_dic, cand ) - visited
        
        connected_comp.append( visited )
        for x in connected_comp:
            covered |= x
    
    return connected_comp

In [18]:
rxn_strs = '[CH3:21][C:22]1([CH3:24])[O:23][B:29]([O:28][C:25]1([CH3:26])[CH3:27])[C:30]2=[CH:31][CH:32]=[CH:33][CH:34]=[CH:35]2.[CH3:9][CH:8]([CH3:10])[N:7]([CH:11]([CH3:12])[CH3:13])[C:6](=[O:14])[O:5][C@H:4]([C:15]3=[CH:16][CH:17]=[CH:18][CH:19]=[CH:20]3)[CH2:3][CH:2]=[CH2:1]>>[CH3:9][CH:8]([CH3:10])[NH:7][CH:11]([CH3:12])[CH3:13].[CH3:21][C:22]1([CH3:24])[O:23][B:29]([O:28][C:25]1([CH3:26])[CH3:27])[C:4]([C:30]=2[CH:35]=[CH:34][CH:33]=[CH:32][CH:31]=2)([C:15]=3[CH:20]=[CH:19][CH:18]=[CH:17][CH:16]=3)[CH2:3][CH:2]=[CH2:1].[O:5]=[C:6]=[O:14]'
rct, prd = rxn_strs.split('>>')
mol_rct = smiles( rct )
mol_prd = smiles( prd )
cgr = mol_rct ^ mol_prd
cgr.clean2d()
cgr

In [16]:
rxn_strs = '[CH3:8][CH2:7][C:6](=[O:9])[CH2:5][C:3](=[O:4])[O:2][CH2:1][CH3:10].[CH:13]1=[CH:12][CH:11]=[C:16]([CH:15]=[CH:14]1)[OH:17]>>[CH3:8][CH2:7][C:6]1=[CH:5][C:3](=[O:4])[O:2][C:1]2=[CH:10][CH:14]=[CH:13][CH:12]=[C:11]21.[CH3:15][CH2:16][OH:17].[OH2:9]'
rct, prd = rxn_strs.split('>>')
mol_rct = smiles( rct )
mol_prd = smiles( prd )
cgr = mol_rct ^ mol_prd
cgr.clean2d()
cgr

In [9]:
with open( 'typical_smarts_2024_07_03.txt', 'r' ) as f:
    strns = [ x.strip() for x in f.readlines() ]

print( strns[0] )

[CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]


In [10]:
import ast

with open(  'partial_mappings_semicolon_delimintor_fixed.csv', 'r' ) as f:
    strn = [ x.strip('\n').split(';') for x in  f.readlines()[1:] ]

dats = []
for x, y in strn:
    if x != '40':
        dats.append( (int(x), ast.literal_eval(y)) )
    else:
        dats.append( (int(x), []) )

In [11]:
res = dict()
for i, x in enumerate( strns ):
    print( '#', i+1, '   reaction' )
    if i == 39:
        continue
    
    RM = ReactionMap( x )
    idx, maps = dats[i]
    print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
    
    complete_maps = RM.permutation_complete( maps )
    h_maps = RM.score( complete_maps, mode='hydrogen' )
    b_maps = RM.score( complete_maps, mode='bond' )
    hb_maps = RM.score( h_maps, mode='bond' )
    bh_maps = RM.score( b_maps, mode='hydrogen' )
    
    print( 'complete mappings                 : ', len( complete_maps ) )
    print( 'selected mappings (hydrogen)      : ', len( h_maps ) )
    print( 'selected mappings (bond)          : ', len( b_maps ) )
    print( 'selected mappings (hydrogen, bond): ', len( hb_maps ) )
    print( 'selected mappings (bond, hydrogen): ', len( bh_maps ) )
    
    l1 = len( complete_maps )
    l2 = len( h_maps )
    l3 = len( b_maps )
    l4 = len( hb_maps )
    l5 = len( bh_maps )
    
    with open( 'result_0807.csv', 'a' ) as f:
        f.write( str(l1) + ',' + str(l2) + ',' +str(l3) + ',' +str(l4) + ',' +str(l5) + ',' )
    
        c_res = RM.check_answer( complete_maps )
        h_res = RM.check_answer( h_maps )
        b_res = RM.check_answer( b_maps )
        hb_res = RM.check_answer( hb_maps )
        bh_res = RM.check_answer( hb_maps )
        
        if c_res != None:
            print( 'c-CORRECT' )
            f.write( str(1) + ',' )
        else:
            print( 'c-WRONG' )
            f.write( str(0) + ',' )
        
        if h_res != None:
            print( 'h-CORRECT' )
            f.write( str(1) + ',' )
        else:
            print( 'h-WRONG' )
            f.write( str(0) + ',' )
        
        if b_res != None:
            print( 'b-CORRECT' )
            f.write( str(1) + ',' )
        else:
            print( 'b-WRONG' )
            f.write( str(0) + ',' )
        
        if hb_res != None:
            print( '(hb)-CORRECT' )
            f.write( str(1) + ',' )
        else:
            print( '(hb)-WRONG' )
            f.write( str(0) + ',' )
        
        if bh_res != None:
            print( '(bh)-CORRECT' )
            f.write( str(1) + ',' )
        else:
            print( '(bh)-WRONG' )
            f.write( str(0) + ',' )
        
        print( 'non-eq mappings                 : ', len( RM.isomorphism( complete_maps, mode='skelton' )))
        print( 'non-eq mappings (hydrogen)      : ', len( RM.isomorphism( h_maps, mode='skelton' )))
        print( 'non-eq mappings (bond)          : ', len( RM.isomorphism( b_maps, mode='skelton' )))
        print( 'non-eq mappings (hydrogen, bond): ', len( RM.isomorphism( hb_maps, mode='skelton' )))
        print( 'non-eq mappings (bond, hydrogen): ', len( RM.isomorphism( bh_maps, mode='skelton' )))
        print( '\n' )
        
        l11 = len( RM.isomorphism( complete_maps, mode='skelton' ))
        l22 = len( RM.isomorphism( h_maps, mode='skelton' ))
        l33 = len( RM.isomorphism( b_maps, mode='skelton' ))
        l44 = len( RM.isomorphism( hb_maps, mode='skelton' ))
        l55 = len( RM.isomorphism( bh_maps, mode='skelton' ))
        
        f.write( str(l11) + ',' + str(l22) + ',' +str(l33) + ',' + str(l44) + ',' + str(l55) + '\n' )

# 1    reaction
target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.034198 sec
15 / 16 atoms were mapped..
complete mappings                 :  6
selected mappings (hydrogen)      :  6
selected mappings (bond)          :  6
selected mappings (hydrogen, bond):  6
selected mappings (bond, hydrogen):  6
c-CORRECT
h-CORRECT
b-CORRECT
(hb)-CORRECT
(bh)-CORRECT
non-eq mappings                 :  1
non-eq mappings (hydrogen)      :  1
non-eq mappings (bond)          :  1
non-eq mappings (hydrogen, bond):  1
non-eq mappings (bond, hydrogen):  1


# 2    reaction
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([O

complete mappings                 :  2048
selected mappings (hydrogen)      :  2048
selected mappings (bond)          :  1024
selected mappings (hydrogen, bond):  1024
selected mappings (bond, hydrogen):  1024
c-CORRECT
h-CORRECT
b-CORRECT
(hb)-CORRECT
(bh)-CORRECT


KeyboardInterrupt: 

In [35]:
RM.RXN.__format__('smiles')

'[CH3:3][C:2](=[O:1])[O:4][C:5]([CH3:6])=[O:7].[N:22]([OH:23])=[C:14]([CH2:15][C:16]1=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]1)[C:13]=2[CH:12]=[CH:11][CH:10]=[C:9]([CH:24]=2)[Br:8]>>[OH:4][C:5]([CH3:6])=[O:7].[OH:23][C:2]([CH3:3])=[O:1].[CH:24]=1[C:9]([Br:8])=[CH:10][CH:11]=[CH:12][C:13]=1[C:14]2=[N:22][CH:15]2[C:16]3=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]3'

In [ ]:
only_visiualization( maps, RM, rxn_str ):

In [36]:
RM.RXN.__format__('s')

'CC(=O)OC(C)=O.N(O)=C(CC1=CC=CC=N1)C=2C=CC=C(C=2)Br>>OC(C)=O.OC(C)=O.C=1C(Br)=CC=CC=1C2=NC2C3=CC=CC=N3'

In [28]:
import ast

with open( 'partial_mappings_modified_case-40.csv', 'r' ) as f:
    strn = [ x.strip('\n') for x in  f.readlines()[1:] ]
strn

['[(1, 23), (2, 2), (3, 3), (4, 1), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (24, 24)]',
 '[(1, 4), (2, 5), (3, 6), (4, 7), (5, 2), (6, 3), (7, 23), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (24, 24)]',
 '[(1, 4), (2, 5), (3, 6), (4, 23), (5, 2), (6, 3), (7, 1), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (24, 24)]',
 '[(1, 23), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (24, 24)]',
 '[(1, 7), (2, 5), (3, 6), (4, 4), (5, 2), (6, 3), (7, 23), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (1

In [30]:
data = []
for x in strn:
    data.append( ast.literal_eval(x) )
data

[[(1, 23),
  (2, 2),
  (3, 3),
  (4, 1),
  (5, 5),
  (6, 6),
  (7, 7),
  (8, 8),
  (9, 9),
  (10, 10),
  (11, 11),
  (12, 12),
  (13, 13),
  (14, 14),
  (15, 15),
  (16, 16),
  (17, 17),
  (18, 18),
  (19, 19),
  (20, 20),
  (21, 21),
  (22, 22),
  (24, 24)],
 [(1, 4),
  (2, 5),
  (3, 6),
  (4, 7),
  (5, 2),
  (6, 3),
  (7, 23),
  (8, 8),
  (9, 9),
  (10, 10),
  (11, 11),
  (12, 12),
  (13, 13),
  (14, 14),
  (15, 15),
  (16, 16),
  (17, 17),
  (18, 18),
  (19, 19),
  (20, 20),
  (21, 21),
  (22, 22),
  (24, 24)],
 [(1, 4),
  (2, 5),
  (3, 6),
  (4, 23),
  (5, 2),
  (6, 3),
  (7, 1),
  (8, 8),
  (9, 9),
  (10, 10),
  (11, 11),
  (12, 12),
  (13, 13),
  (14, 14),
  (15, 15),
  (16, 16),
  (17, 17),
  (18, 18),
  (19, 19),
  (20, 20),
  (21, 21),
  (22, 22),
  (24, 24)],
 [(1, 23),
  (2, 2),
  (3, 3),
  (4, 4),
  (5, 5),
  (6, 6),
  (7, 7),
  (8, 8),
  (9, 9),
  (10, 10),
  (11, 11),
  (12, 12),
  (13, 13),
  (14, 14),
  (15, 15),
  (16, 16),
  (17, 17),
  (18, 18),
  (19, 19),
  (20, 20

In [32]:
RM = ReactionMap( '[O:1]=[C:2]([O:4][C:5]([CH3:6])=[O:7])[CH3:3].[Br:8][C:9]1=[CH:10][CH:11]=[CH:12][C:13]([C:14]([CH2:15][C:16]2=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]2)=[N:22][OH:23])=[CH:24]1>>[CH3:3][C:2]([OH:23])=[O:1].[CH3:6][C:5]([OH:4])=[O:7].[Br:8][C:9]3=[CH:10][CH:11]=[CH:12][C:13]([C:14]4=[N:22][CH:15]4[C:16]5=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]5)=[CH:24]3' )
maps = data
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )

complete_maps = RM.permutation_complete( maps )
h_maps = RM.score( complete_maps, mode='hydrogen' )
b_maps = RM.score( complete_maps, mode='bond' )
hb_maps = RM.score( h_maps, mode='bond' )
bh_maps = RM.score( b_maps, mode='hydrogen' )

print( 'complete mappings                 : ', len( complete_maps ) )
print( 'selected mappings (hydrogen)      : ', len( h_maps ) )
print( 'selected mappings (bond)          : ', len( b_maps ) )
print( 'selected mappings (hydrogen, bond): ', len( hb_maps ) )
print( 'selected mappings (bond, hydrogen): ', len( bh_maps ) )

l1 = len( complete_maps )
l2 = len( h_maps )
l3 = len( b_maps )
l4 = len( hb_maps )
l5 = len( bh_maps )

with open( 'simplified_case_40.csv', 'a' ) as f:
    f.write( str(l1) + ',' + str(l2) + ',' +str(l3) + ',' +str(l4) + ',' +str(l5) + ',' )
    c_res = RM.check_answer( complete_maps )
    h_res = RM.check_answer( h_maps )
    b_res = RM.check_answer( b_maps )
    hb_res = RM.check_answer( hb_maps )
    bh_res = RM.check_answer( hb_maps )
    
    if c_res != None:
        print( 'c-CORRECT' )
        f.write( str(1) + ',' )
    else:
        print( 'c-WRONG' )
        f.write( str(0) + ',' )
    
    if h_res != None:
        print( 'h-CORRECT' )
        f.write( str(1) + ',' )
    else:
        print( 'h-WRONG' )
        f.write( str(0) + ',' )
    
    if b_res != None:
        print( 'b-CORRECT' )
        f.write( str(1) + ',' )
    else:
        print( 'b-WRONG' )
        f.write( str(0) + ',' )
    
    if hb_res != None:
        print( '(hb)-CORRECT' )
        f.write( str(1) + ',' )
    else:
        print( '(hb)-WRONG' )
        f.write( str(0) + ',' )
    
    if bh_res != None:
        print( '(bh)-CORRECT' )
        f.write( str(1) + ',' )
    else:
        print( '(bh)-WRONG' )
        f.write( str(0) + ',' )
    
    print( 'non-eq mappings                 : ', len( RM.isomorphism( complete_maps, mode='skelton' )))
    print( 'non-eq mappings (hydrogen)      : ', len( RM.isomorphism( h_maps, mode='skelton' )))
    print( 'non-eq mappings (bond)          : ', len( RM.isomorphism( b_maps, mode='skelton' )))
    print( 'non-eq mappings (hydrogen, bond): ', len( RM.isomorphism( hb_maps, mode='skelton' )))
    print( 'non-eq mappings (bond, hydrogen): ', len( RM.isomorphism( bh_maps, mode='skelton' )))
    print( '\n' )
    
    l11 = len( RM.isomorphism( complete_maps, mode='skelton' ))
    l22 = len( RM.isomorphism( h_maps, mode='skelton' ))
    l33 = len( RM.isomorphism( b_maps, mode='skelton' ))
    l44 = len( RM.isomorphism( hb_maps, mode='skelton' ))
    l55 = len( RM.isomorphism( bh_maps, mode='skelton' ))
    
    f.write( str(l11) + ',' + str(l22) + ',' +str(l33) + ',' + str(l44) + ',' + str(l55) + '\n' )

target reaction: [O:1]=[C:2]([O:4][C:5]([CH3:6])=[O:7])[CH3:3].[Br:8][C:9]1=[CH:10][CH:11]=[CH:12][C:13]([C:14]([CH2:15][C:16]2=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]2)=[N:22][OH:23])=[CH:24]1>>[CH3:3][C:2]([OH:23])=[O:1].[CH3:6][C:5]([OH:4])=[O:7].[Br:8][C:9]3=[CH:10][CH:11]=[CH:12][C:13]([C:14]4=[N:22][CH:15]4[C:16]5=[CH:17][CH:18]=[CH:19][CH:20]=[N:21]5)=[CH:24]3
# initialization: 
	>> time:  0.013104 sec
23 / 24 atoms were mapped..
complete mappings                 :  15
selected mappings (hydrogen)      :  7
selected mappings (bond)          :  3
selected mappings (hydrogen, bond):  3
selected mappings (bond, hydrogen):  3
c-CORRECT
h-CORRECT
b-CORRECT
(hb)-CORRECT
(bh)-CORRECT
non-eq mappings                 :  4
non-eq mappings (hydrogen)      :  2
non-eq mappings (bond)          :  1
non-eq mappings (hydrogen, bond):  1
non-eq mappings (bond, hydrogen):  1




In [12]:
with open( 'dataset_typical_241_reactions.txt', 'r' ) as f:
    strn = [ x.strip('\n') for x in f.readlines() ]

with open( 'no_mapping_information_241_reactions.txt', 'w' ) as f2:
    for x in strn:
        y, z = x.split('>>')
        rct, prd = smiles( y ), smiles( z )
        rxn = ReactionContainer( [rct], [prd] )
        f2.write( rxn.__format__('s') + '\n' )

In [38]:
i = 22 - 1
x = strns[i]
RM = ReactionMap( x )
idx, maps = dats[i]
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
complete_maps = RM.permutation_complete( maps )
hyd_maps = RM.score( complete_maps, mode='hydrogen' )
hyd_bn_maps = RM.score( hyd_maps, mode='bond' )
print( 'complete mappings                 : ', len( complete_maps ) )
print( 'selected mappings (hydrogen)      : ', len( hyd_maps ) )
print( 'selected mappings (hydrogen, bond): ', len( hyd_bn_maps ) )
c_res = RM.check_answer( complete_maps )
h_res = RM.check_answer( hyd_maps )
b_res = RM.check_answer( hyd_bn_maps )
if c_res != None:
    print( 'c-CORRECT' )
else:
    print( 'c-WRONG' )
if h_res != None:
    print( 'h-CORRECT' )
else:
    print( 'h-WRONG' )
if b_res != None:
    print( 'b-CORRECT' )
else:
    print( 'b-WRONG' )
print( 'non-eq mappings                 : ', len( RM.isomorphism( complete_maps, mode='skelton' )))
print( 'non-eq mappings (hydrogen)      : ', len( RM.isomorphism( hyd_maps, mode='skelton' )))
print( 'non-eq mappings (hydrogen, bond): ', len( RM.isomorphism( hyd_bn_maps, mode='skelton' )))

target reaction: [Cl:15][C:14]([Cl:16])([Cl:17])[CH:10]([O:11][CH2:12][CH3:13])[OH:9].[OH:3][C:2]([C@@H:1]1[CH2:5][CH2:6][CH2:7][NH:8]1)=[O:4]>>[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:10]1[O:9][C:2]([C@H:1]2[N:8]1[CH2:7][CH2:6][CH2:5]2)=[O:4].[OH2:3].[CH2:12]([CH3:13])[OH:11]
# initialization: 
	>> time:  0.018063 sec
16 / 17 atoms were mapped..
complete mappings                 :  24
selected mappings (hydrogen)      :  18
selected mappings (hydrogen, bond):  12
c-CORRECT
h-CORRECT
b-CORRECT
non-eq mappings                 :  4
non-eq mappings (hydrogen)      :  3
non-eq mappings (hydrogen, bond):  2


In [39]:
selected_maps = RM.isomorphism( hyd_bn_maps, mode='skelton' )
selected_num = [ x.pop() for x in selected_maps ]
selected_num

[0, 2]

In [40]:
RM.RXN

In [41]:
i = 0
hyd_bn_maps[i].sort()
[ (x,y) for x,y in hyd_bn_maps[i] if x!=y ]

[(15, 16), (16, 17), (17, 15)]

In [42]:
i = 2
hyd_bn_maps[i].sort()
[ (x,y) for x,y in hyd_bn_maps[i] if x!=y ]

[(3, 9), (9, 3), (15, 16), (16, 15)]

In [45]:
RM.RXN

In [46]:
R = deepcopy( RM.RCT )
R.remap( dict( hyd_bn_maps[0] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [51]:
R = deepcopy( RM.RCT )
R.remap( dict( hyd_bn_maps[2] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [27]:
R = deepcopy( RM.RCT )
R.remap( dict( hyd_bn_maps[12] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [26]:
R = deepcopy( RM.RCT )
R.remap( dict( hyd_bn_maps[13] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [24]:
only_visiualization( hyd_bn_maps[12], RM, x )

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [60]:
res = dict()
for i, x in enumerate( strns ):
    print( '#', i+1, '   reaction' )
    if i == 39:
        continue
    
    RM = ReactionMap( x )
    idx, maps = dats[i]
    print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
    
    maps_comp = RM.permutation_complete( maps )
    print( 'complete mappings: ', len( maps_comp ) )
    
    mps_sk = RM.isomorphism( maps_comp, mode='skelton' )
    print( 'independent mappings (skelton): ', len( mps_sk ) )
    
    mps_bn = RM.isomorphism( maps_comp, mode='bond' )
    print( 'independent mappings (bond)   : ', len( mps_bn ) )
    
    selected_mps = [ maps_comp[t] for t in [ x.pop() for x in mps_bn ] ]
    res = RM.check_answer( selected_mps )
    
    if res != None:
        print( 'correct map: ', selected_mps.index( res ) )
        print( '>>> mapping is CORRECT' )
        
        min_sc = 999
        for mp in selected_mps:
            sc = sum( [ abs( RM.rct.dic_hyd[r] - RM.prd.dic_hyd[p] ) for r, p in mp ] )
            if sc < min_sc:
                min_sc = sc
            print( selected_mps.index( mp ), sc )
        
        ans_sc = sum( [ abs( RM.rct.dic_hyd[r] - RM.prd.dic_hyd[p] ) for r, p in res ] )
        if ans_sc > min_sc:
            print( '### NOT minimum hydrogen score was detected' )
        
    else:
        print( '>>> mapping is WRONG' )

    h_maps = RM.score( maps_comp, mode='hydrogen' )
    print( 'Hs min complete mappings: ', len( h_maps ) )
    hmps_sk = RM.isomorphism( h_maps, mode='skelton' )
    print( 'Hs min independent mappings (skelton): ', len( hmps_sk ) )
    hmps_bn = RM.isomorphism( h_maps, mode='bond' )
    print( 'Hs min independent mappings (bond)   : ', len( hmps_bn ) )
    
    h_selected_mps = [ h_maps[t] for t in [ x.pop() for x in hmps_bn ] ]
    h_res = RM.check_answer( h_selected_mps )
    
    if h_res != None:
        print( 'correct map: ', h_selected_mps.index( h_res ) )
        print( '>>>> mapping is h-CORRECT\n' )
    else:
        print( '>>>> mapping is h-WRONG\n' )

# 1    reaction
target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.043358 sec
15 / 16 atoms were mapped..
complete mappings:  6
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  6
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 2    reaction
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([OH:2])[C:14]=1[OH:15])[O:6][CH3:5]
# initialization: 
	>> time:  0.009973 sec
11 / 15 atoms were mapped..
complete mappings:  48
independent mapping

independent mappings (skelton):  1
independent mappings (bond)   :  4
correct map:  0
>>> mapping is CORRECT
0 4
1 4
2 4
3 4
Hs min complete mappings:  2048
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  4
correct map:  0
>>>> mapping is h-CORRECT

# 15    reaction
target reaction: [CH3:22][C:23]([CH3:24])([CH3:29])[Si:25]([CH3:26])([CH3:28])[Cl:27].[CH2:15]1[CH2:16][C@:17]2([C:12](=[CH:11][CH2:10][C@@H:9]3[C@@H:8]2[CH2:7][CH2:6][C@:5]4([CH3:20])[C:4]([CH2:3][CH2:2][C@@H:1]34)=[O:21])[CH2:13][C@H:14]1[OH:19])[CH3:18].[CH:33]5=[CH:32][N:31]=[CH:30][NH:34]5>>[CH3:22][C:23]([CH3:24])([CH3:29])[Si:25]([CH3:26])([CH3:28])[O:19][C@@H:14]1[CH2:13][C:12]=2[C@:17]([CH3:18])([CH2:16][CH2:15]1)[C@@H:8]3[C@@H:9]([CH2:10][CH:11]=2)[C@H:1]4[C@:5]([CH3:20])([CH2:6][CH2:7]3)[C:4]([CH2:3][CH2:2]4)=[O:21].[Cl-:27].[CH:30]5=[NH+:31][CH:32]=[CH:33][NH:34]5
# initialization: 
	>> time:  0.129136 sec
33 / 34 atoms were mapped..
complete mappings:  24
independent mappings (

# initialization: 
	>> time:  0.005984 sec
11 / 13 atoms were mapped..
complete mappings:  4
independent mappings (skelton):  1
independent mappings (bond)   :  2
correct map:  0
>>> mapping is CORRECT
0 0
1 0
Hs min complete mappings:  4
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  2
correct map:  0
>>>> mapping is h-CORRECT

# 27    reaction
target reaction: [CH2:18]1[CH2:17][CH:16]([CH2:21][CH2:20][CH2:19]1)[N-:22][CH:23]2[CH2:24][CH2:25][CH2:26][CH2:27][CH2:28]2.[CH3:13][CH:12]([CH3:14])[C:11]([O:10][CH3:9])=[O:15].[CH:6]=3[C:5]([Br:8])=[CH:4][CH:3]=[C:2]([Cl:1])[CH:7]=3>>[CH2:18]1[CH2:17][CH:16]([CH2:21][CH2:20][CH2:19]1)[NH:22][CH:23]2[CH2:24][CH2:25][CH2:26][CH2:27][CH2:28]2.[CH3:13][C:12]([CH3:14])([C:5]=3[CH:4]=[CH:3][C:2](=[CH:7][CH:6]=3)[Cl:1])[C:11]([O:10][CH3:9])=[O:15].[Br-:8]
# initialization: 
	>> time:  0.039705 sec
27 / 28 atoms were mapped..
complete mappings:  32
independent mappings (skelton):  1
independent mappings (bond)   : 

# initialization: 
	>> time:  0.024537 sec
23 / 25 atoms were mapped..
complete mappings:  32
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  32
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 49    reaction
target reaction: [CH2:25]1[CH2:26][CH2:27][CH2:28][N:24]1[C:23](=[O:29])[C:22]=2[C:14]([CH:13]=[CH:12][C:9]=3[CH:10]=[CH:11][C:6](=[CH:7][CH:8]=3)[F:5])=[C:15]([CH:19]=[CH:20][CH:21]=2)[N:16](=[O:17])=[O:18].[C-:1]#[O+:2].[C-:3]#[O+:4]>>[O:2]=[C:1]=[O:17].[O:4]=[C:3]=[O:18].[CH2:25]1[CH2:26][CH2:27][CH2:28][N:24]1[C:23]([C:22]=2[C:14]3=[C:15]([CH:19]=[CH:20][CH:21]=2)[NH:16][C:12]([C:9]4=[CH:8][CH:7]=[C:6]([F:5])[CH:11]=[CH:10]4)=[CH:13]3)=[O:29]
# initialization: 
	>> time:  0.01496 sec
27 / 29 atoms were mapped..
complete mappings:  64
independent mappings (skelton):  1
independent mappings (bond)   :  2
corre

# initialization: 
	>> time:  0.010972 sec
19 / 20 atoms were mapped..
complete mappings:  72
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  72
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 63    reaction
target reaction: [CH:17]#[CH:18].[O:5]=[C:4]([CH:6]1[C:15](=[O:16])[C:10]2=[C:9]([CH:14]=[CH:13][CH:12]=[CH:11]2)[CH2:8][CH2:7]1)[O:3][CH2:2][CH3:1]>>[CH:12]1=[CH:13][CH:14]=[C:9]2[CH2:8][CH2:7][C:6]([C:4]([O:3][CH2:2][CH3:1])=[O:5])([C:15](=[O:16])[C:10]2=[CH:11]1)[CH:17]=[CH2:18]
# initialization: 
	>> time:  0.012965 sec
16 / 18 atoms were mapped..
complete mappings:  2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  2
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>

...

# initialization: 
	>> time:  0.027925 sec
15 / 17 atoms were mapped..
complete mappings:  8
independent mappings (skelton):  1
independent mappings (bond)   :  2
correct map:  0
>>> mapping is CORRECT
0 4
1 4
Hs min complete mappings:  8
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  2
correct map:  0
>>>> mapping is h-CORRECT

# 143    reaction
target reaction: [CH:7]1=[CH:8][CH2:6]1.[CH:2]=2[CH:3]=[CH:4][CH2:5][CH:1]=2>>[CH:1]12[CH:7]3[CH:8]([CH:4]([CH:3]=[CH:2]1)[CH2:5]2)[CH2:6]3
# initialization: 
	>> time:  0.045627 sec
8 / 8 atoms were mapped..
complete mappings:  60
independent mappings (skelton):  8
independent mappings (bond)   :  8
correct map:  6
>>> mapping is CORRECT
0 4
1 4
2 2
3 4
4 4
5 2
6 0
7 2
Hs min complete mappings:  4
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 144    reaction
target reaction: [CH:16]1=[CH:17][C:10]([NH2:9])=[CH:11][CH:12]=[C:13

# initialization: 
	>> time:  0.015703 sec
24 / 25 atoms were mapped..
complete mappings:  8
independent mappings (skelton):  1
independent mappings (bond)   :  8
correct map:  0
>>> mapping is CORRECT
0 2
1 2
2 2
3 2
4 2
5 2
6 2
7 2
Hs min complete mappings:  8
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  8
correct map:  0
>>>> mapping is h-CORRECT

# 169    reaction
target reaction: [CH3:9][C:10](=[O:15])[O:11][C:12]([CH3:13])=[O:14].[CH2:5]1[CH2:4][CH2:3][CH:2]([C:7](=[O:8])[CH2:6]1)[CH3:1]>>[CH2:3]1[C:2]([CH3:1])=[C:7]([CH2:6][CH2:5][CH2:4]1)[O:8][C:12](=[O:14])[CH3:13].[OH:11][C:10]([CH3:9])=[O:15]
# initialization: 
	>> time:  0.006981 sec
15 / 15 atoms were mapped..
complete mappings:  4
independent mappings (skelton):  2
independent mappings (bond)   :  2
correct map:  1
>>> mapping is CORRECT
0 2
1 2
Hs min complete mappings:  4
Hs min independent mappings (skelton):  2
Hs min independent mappings (bond)   :  2
correct map:  1
>>>> mapping 

# initialization: 
	>> time:  0.013688 sec
20 / 21 atoms were mapped..
complete mappings:  16
independent mappings (skelton):  1
independent mappings (bond)   :  2
correct map:  0
>>> mapping is CORRECT
0 2
1 2
Hs min complete mappings:  16
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  2
correct map:  0
>>>> mapping is h-CORRECT

# 197    reaction
target reaction: [CH3:21][C:22]1([CH3:24])[O:23][B:29]([O:28][C:25]1([CH3:26])[CH3:27])[C:30]2=[CH:31][CH:32]=[CH:33][CH:34]=[CH:35]2.[CH3:9][CH:8]([CH3:10])[N:7]([CH:11]([CH3:12])[CH3:13])[C:6](=[O:14])[O:5][C@H:4]([C:15]3=[CH:16][CH:17]=[CH:18][CH:19]=[CH:20]3)[CH2:3][CH:2]=[CH2:1]>>[CH3:9][CH:8]([CH3:10])[NH:7][CH:11]([CH3:12])[CH3:13].[CH3:21][C:22]1([CH3:24])[O:23][B:29]([O:28][C:25]1([CH3:26])[CH3:27])[C:4]([C:30]=2[CH:35]=[CH:34][CH:33]=[CH:32][CH:31]=2)([C:15]=3[CH:20]=[CH:19][CH:18]=[CH:17][CH:16]=3)[CH2:3][CH:2]=[CH2:1].[O:5]=[C:6]=[O:14]
# initialization: 
	>> time:  0.022938 sec
35 / 35 atoms we

# initialization: 
	>> time:  0.004027 sec
8 / 8 atoms were mapped..
complete mappings:  2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 0
Hs min complete mappings:  2
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 220    reaction
target reaction: [CH3:11][C:12]([CH3:13])([CH3:14])[O-:15].[CH3:1][CH:2]=[O:3].[CH3:4][CH2:5][O:6][C:7]([CH2:9][Cl:10])=[O:8]>>[CH3:11][C:12]([CH3:13])([CH3:14])[OH:15].[O:8]=[C:7]([CH:9]1[O:3][CH:2]1[CH3:1])[O:6][CH2:5][CH3:4].[Cl-:10]
# initialization: 
	>> time:  0.006981 sec
14 / 15 atoms were mapped..
complete mappings:  6
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  6
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 221    reaction
targe

# initialization: 
	>> time:  0.07467 sec
37 / 37 atoms were mapped..
complete mappings:  16
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 2
Hs min complete mappings:  16
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> mapping is h-CORRECT

# 234    reaction
target reaction: [O:11]=[C:10]([O:9][CH2:8][CH3:7])[C:12]([C:14](=[O:15])[O:16][CH2:17][CH3:18])=[O:13].[CH3:1][O:2]/[CH:3]=[CH:4]/[CH:5]=[CH2:6]>>[CH2:8]([CH3:7])[O:9][C:10](=[O:11])[C:12]1([C:14]([O:16][CH2:17][CH3:18])=[O:15])[O:13][CH:3]([O:2][CH3:1])[CH:4]=[CH:5][CH2:6]1
# initialization: 
	>> time:  0.007979 sec
18 / 18 atoms were mapped..
complete mappings:  2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT
0 0
Hs min complete mappings:  2
Hs min independent mappings (skelton):  1
Hs min independent mappings (bond)   :  1
correct map:  0
>>>> ma

In [124]:
i = 90 - 1
x = strns[i]

RM = ReactionMap( x )
idx, maps = dats[i]
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
maps_comp = RM.permutation_complete( maps )
print( 'complete mappings: ', len( maps_comp ) )
mps_sk = RM.isomorphism( maps_comp, mode='skelton' )
print( 'independent mappings (skelton): ', len( mps_sk ) )
mps_bn = RM.isomorphism( maps_comp, mode='bond' )
print( 'independent mappings (bond)   : ', len( mps_bn ) )
selected_mps = [ maps_comp[t] for t in [ x.pop() for x in mps_bn ] ]
res = RM.check_answer( selected_mps )
if res != None:
    print( 'correct map: ', selected_mps.index( res ) )
    print( '>>> mapping is CORRECT' )
    
    min_sc = 999
    for mp in selected_mps:
        sc = sum( [ abs( RM.rct.dic_hyd[r] - RM.prd.dic_hyd[p] ) for r, p in mp ] )
        if sc < min_sc:
            min_sc = sc
        print( selected_mps.index( mp ), sc )
    
    ans_sc = sum( [ abs( RM.rct.dic_hyd[r] - RM.prd.dic_hyd[p] ) for r, p in res ] )
    if ans_sc > min_sc:
        print( '### NOT minimum hydrogen score was detected' )
    
else:
    print( '>>> mapping is WRONG' )
h_maps = RM.score( maps_comp, mode='hydrogen' )
print( 'Hs min complete mappings: ', len( h_maps ) )
hmps_sk = RM.isomorphism( h_maps, mode='skelton' )
print( 'Hs min independent mappings (skelton): ', len( hmps_sk ) )
hmps_bn = RM.isomorphism( h_maps, mode='bond' )
print( 'Hs min independent mappings (bond)   : ', len( hmps_bn ) )

res = RM.check_answer( maps_comp )
if h_res != None:
    print( 'correct map: ', maps_comp.index( res ) )
    print( '>>>> mapping is h-CORRECT\n' )
else:
    print( '>>>> mapping is h-WRONG\n' )
    
res = RM.check_answer( h_maps )
if h_res != None:
    print( 'correct map: ', h_maps.index( res ) )
    print( '>>>> mapping is h-CORRECT\n' )
else:
    print( '>>>> mapping is h-WRONG\n' )


target reaction: [OH2:19].[OH2:20].[CH2:2]([CH3:1])[O:3][C:4]([CH:5]1[C:15]([CH2:14][CH:8]([C:7]([CH2:6]1)=[O:17])[C:9]([O:10][CH2:11][CH3:12])=[O:13])=[O:16])=[O:18]>>[O:13]=[C:9]=[O:19].[O:18]=[C:4]=[O:20].[CH2:5]1[CH2:6][C:7]([CH2:8][CH2:14][C:15]1=[O:16])=[O:17].[CH2:2]([OH:3])[CH3:1].[CH2:11]([OH:10])[CH3:12]
# initialization: 
	>> time:  0.009849 sec
14 / 20 atoms were mapped..
complete mappings:  3712
independent mappings (skelton):  11
independent mappings (bond)   :  11
>>> mapping is WRONG
Hs min complete mappings:  512
Hs min independent mappings (skelton):  2
Hs min independent mappings (bond)   :  2
>>>> mapping is h-WRONG

>>>> mapping is h-WRONG



In [119]:
cgr = RM.RCT ^ RM.PRD
cgr.clean2d()
cgr

In [120]:
R = deepcopy( RM.RCT )
selected_mps = [ h_maps[t] for t in [ x.pop() for x in hmps_sk ] ]
R.remap( dict( selected_mps[0] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [121]:
R = deepcopy( RM.RCT )
R.remap( dict( selected_mps[1] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [117]:
R = deepcopy( RM.RCT )
R.remap( dict( selected_mps[9] ) )
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [144]:
res = dict()
for i, x in enumerate( strns ):
    print( '#', i+1, '   reaction' )
    if i == 39:
        continue
    
    RM = ReactionMap( x )
    idx, maps = dats[i]
    print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
    
    maps_comp = RM.permutation_complete( maps )
    print( 'complete mappings: ', len( maps_comp ) )
    res = RM.check_answer( maps_comp )
    
    if res != None:
        print( 'correct map: ', maps_comp.index( res ) )
        print( '>>> mapping is CORRECT' )   
    else:
        print( '>>> mapping is WRONG' )

# 1    reaction
target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.0085 sec
15 / 16 atoms were mapped..
complete mappings:  6
correct map:  0
>>> mapping is CORRECT
# 2    reaction
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([OH:2])[C:14]=1[OH:15])[O:6][CH3:5]
# initialization: 
	>> time:  0.006079 sec
11 / 15 atoms were mapped..
complete mappings:  48
correct map:  0
>>> mapping is CORRECT
# 3    reaction
target reaction: [O:1]=[N:2](=[O:12])[C:3]=1[C:8]([N:9]=[N+:10]=[N-:11])=[CH:7][CH:6]=[CH:5][CH:4]=1>>[N:10]#[N:11].[C:8]1=2[C:3]([CH:4]=[CH:5][CH:6]=[CH:7]1)=[N+:2]([O:1][N:9]=2)[O-:12]
# initial

# initialization: 
	>> time:  0.02471 sec
27 / 28 atoms were mapped..
complete mappings:  32
correct map:  0
>>> mapping is CORRECT
# 20    reaction
target reaction: [C:10]1(=[O:11])[CH2:12][CH2:13][CH2:14][C:15](=[O:16])[C:6]1([CH2:5][CH2:4][C:2](=[O:3])[CH3:1])[CH2:7][CH:8]=[CH2:9]>>[CH2:4]1[CH2:5][C@:6]2([C:10]([CH2:12][CH2:13][CH2:14][C:15]2=[O:16])=[CH:1][C:2]1=[O:3])[CH2:7][CH:8]=[CH2:9].[OH2:11]
# initialization: 
	>> time:  0.007084 sec
15 / 16 atoms were mapped..
complete mappings:  2
correct map:  0
>>> mapping is CORRECT
# 21    reaction
target reaction: [CH3:1][CH:2]([CH3:3])[N-:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19]([C@H:8]2[N:12]1[CH2:11][CH2:10][CH2:9]2)=[O:20].[CH:23](=[CH2:24])[CH2:22][Br:21]>>[CH3:1][CH:2]([CH3:3])[NH:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19](=[O:20])[C@@:8]2([CH2:9][CH2:10][CH2:11][N:12]12)[CH2:22][CH:23]=[CH2:24].[Br-:21]
# initialization: 
	>> time:  0.015142 sec
23 / 24 ato

# initialization: 
	>> time:  0.020053 sec
29 / 30 atoms were mapped..
complete mappings:  64
correct map:  0
>>> mapping is CORRECT
# 37    reaction
target reaction: [CH:6]=1[CH:5]=[CH:4][CH:3]=[C:2]([CH:1]=[O:8])[CH:7]=1.[C:11]2(=[C:10]([CH:15]=[CH:14][CH:13]=[CH:12]2)[CH3:9])[NH2:16]>>[NH:16]([CH2:1][C:2]=1[CH:3]=[CH:4][CH:5]=[CH:6][CH:7]=1)[C:11]=2[CH:12]=[CH:13][CH:14]=[CH:15][C:10]=2[CH3:9].[OH2:8]
# initialization: 
	>> time:  0.012973 sec
15 / 16 atoms were mapped..
complete mappings:  2
correct map:  0
>>> mapping is CORRECT
# 38    reaction
target reaction: [SH2:14].[SH2:15].[SH2:16].[O:12]=[N:11](=[O:13])[C:3]1=[C:2]([CH:7]=[CH:6][C:5](=[CH:4]1)[N:8](=[O:9])=[O:10])[NH2:1]>>[SH2:14].[SH2:15].[SH2:16].[OH2:12].[OH2:13].[O:9]=[N:8](=[O:10])[C:5]=1[CH:6]=[CH:7][C:2]([NH2:1])=[C:3]([CH:4]=1)[NH2:11]
# initialization: 
	>> time:  0.006983 sec
11 / 16 atoms were mapped..
complete mappings:  24
correct map:  0
>>> mapping is CORRECT
# 39    reaction
target reaction: [CH2:16]1[CH2:1

# initialization: 
	>> time:  0.021798 sec
37 / 39 atoms were mapped..
complete mappings:  192
correct map:  0
>>> mapping is CORRECT
# 61    reaction
target reaction: [Br:15][Br:16].[O:14]=[N+:12]([O-:13])[C:11]=1[C:2]([CH3:1])=[C:3]([CH:8]=[CH:9][CH:10]=1)[C:4](=[O:7])[O:5][CH3:6]>>[BrH:16].[CH:10]=1[CH:9]=[CH:8][C:3](=[C:2]([CH2:1][Br:15])[C:11]=1[N+:12]([O-:13])=[O:14])[C:4](=[O:7])[O:5][CH3:6]
# initialization: 
	>> time:  0.014446 sec
14 / 16 atoms were mapped..
complete mappings:  4
correct map:  0
>>> mapping is CORRECT
# 62    reaction
target reaction: [CH3:1][CH2:2][N:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:13][C:14]([CH3:20])([CH3:15])[Si:16]([CH3:17])([CH3:19])[Cl:18].[CH2:9]([CH3:8])[S:10][CH2:11][OH:12]>>[CH3:13][C:14]([CH3:20])([CH3:15])[Si:16]([CH3:17])([CH3:19])[O:12][CH2:11][S:10][CH2:9][CH3:8].[CH2:2]([CH3:1])[NH+:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[Cl-:18]
# initialization: 
	>> time:  0.013964 sec
19 / 20 atoms were mapped..
complete mappings:  72
correct map:  0
>>> m

# initialization: 
	>> time:  0.022127 sec
22 / 23 atoms were mapped..
complete mappings:  12
correct map:  0
>>> mapping is CORRECT
# 81    reaction
target reaction: [CH2:2]([CH:3]=[CH2:4])[NH2:1].[CH2:6]([CH:7]=[CH2:8])[NH2:5].[C:11]([Cl:12])(=[O:13])[O:10][CH3:9]>>[CH2:8]=[CH:7][CH2:6][NH3+:5].[CH2:2]([CH:3]=[CH2:4])[NH:1][C:11](=[O:13])[O:10][CH3:9].[Cl-:12]
# initialization: 
	>> time:  0.004363 sec
12 / 13 atoms were mapped..
complete mappings:  2
correct map:  0
>>> mapping is CORRECT
# 82    reaction
target reaction: [CH2:10]([CH2:11][CH2:12][CH2:13][CH2:14][C:15]#[C:16][C:17]#[C:18][Br:19])[CH3:9].[CH2:5]([CH:6]=[CH2:7])[NH:4][C:3](=[O:8])[O:2][CH3:1].[CH:23]=1[CH:24]=[N:25][CH:20]=[CH:21][CH:22]=1>>[C:18]([N:4]([CH2:5][CH:6]=[CH2:7])[C:3]([O:2][CH3:1])=[O:8])#[C:17][C:16]#[C:15][CH2:14][CH2:13][CH2:12][CH2:11][CH2:10][CH3:9].[CH:23]=1[CH:24]=[NH+:25][CH:20]=[CH:21][CH:22]=1.[Br-:19]
# initialization: 
	>> time:  0.01184 sec
24 / 25 atoms were mapped..
complete mappings:  4
co

...

# initialization: 
	>> time:  0.014992 sec
37 / 41 atoms were mapped..
complete mappings:  24
correct map:  0
>>> mapping is CORRECT
# 131    reaction
target reaction: [O:24]=[S:16](=[O:25])([C:17]=1[CH:23]=[CH:22][C:20]([CH3:21])=[CH:19][CH:18]=1)[NH:15][C:14]=2[C:9](=[CH:10][CH:11]=[CH:12][CH:13]=2)[C:4]([CH2:5][CH2:6][CH2:7][CH3:8])=[C:2]([F:1])[F:3]>>[O:24]=[S:16](=[O:25])([N:15]1[C:2](=[C:4]([CH2:5][CH2:6][CH2:7][CH3:8])[C:9]2=[CH:10][CH:11]=[CH:12][CH:13]=[C:14]12)[F:1])[C:17]=3[CH:23]=[CH:22][C:20](=[CH:19][CH:18]=3)[CH3:21].[F-:3]
# initialization: 
	>> time:  0.019444 sec
24 / 25 atoms were mapped..
complete mappings:  8
correct map:  0
>>> mapping is CORRECT
# 132    reaction
target reaction: [CH3:3][C:4]12[C:5]([CH3:6])([CH2:8][CH:9]=[CH:10][CH2:11]1)[O:7]2.[Br:1][Br:2]>>[Br:1][CH:10]1[CH:9]([Br:2])[CH2:8][C:5]2([CH3:6])[C:4]([CH3:3])([CH2:11]1)[O:7]2
# initialization: 
	>> time:  0.004987 sec
9 / 11 atoms were mapped..
complete mappings:  4
correct map:  0
>>> mapping is CO

# initialization: 
	>> time:  0.004988 sec
7 / 9 atoms were mapped..
complete mappings:  4
correct map:  2
>>> mapping is CORRECT
# 156    reaction
target reaction: [CH3:1][CH:2]=[O:3].[CH2:4]=[CH:5][C:6](=[O:9])[O:7][CH3:8]>>[O:9]=[C:6]([C:5](=[CH2:4])[CH:2]([OH:3])[CH3:1])[O:7][CH3:8]
# initialization: 
	>> time:  0.003999 sec
9 / 9 atoms were mapped..
complete mappings:  1
correct map:  0
>>> mapping is CORRECT
# 157    reaction
target reaction: [CH3:13][Si:12]([CH3:14])([CH3:15])[C:11]#[CH:10].[CH2:8]([CH3:9])[O:7][C:2](=[O:1])/[C:3]([Br:4])=[CH:5]/[Br:6]>>[CH3:13][Si:12]([CH3:14])([CH3:15])[C:11]#[C:10]/[CH:5]=[C:3](/[C:2]([O:7][CH2:8][CH3:9])=[O:1])[Br:4].[BrH:6]
# initialization: 
	>> time:  0.008181 sec
14 / 15 atoms were mapped..
complete mappings:  6
correct map:  0
>>> mapping is CORRECT
# 158    reaction
target reaction: [C:9]([CH3:8])(=[O:14])[O:10][C:11]([CH3:12])=[O:13].[CH2:3]([O:4][CH2:5][CH2:6][NH2:7])[CH2:2][OH:1]>>[NH:7]([CH2:6][CH2:5][O:4][CH2:3][CH2:2][OH:1])[C:11

# initialization: 
	>> time:  0.031242 sec
16 / 18 atoms were mapped..
complete mappings:  8
correct map:  0
>>> mapping is CORRECT
# 181    reaction
target reaction: [CH3:20][C:19]1([CH3:21])[O:18][CH:17]([CH2:23][O:22]1)[CH2:16][O:15][C:14]([CH2:13][CH2:12][CH2:11][CH2:10][CH2:9]/[CH:8]=[CH:7]\[CH2:6][CH2:5][CH2:4][CH2:3][CH2:2][CH3:1])=[O:24].[OH2:25]>>[CH3:20][C:19]([CH3:21])=[O:25].[CH2:13]([C:14]([O:15][CH2:16][CH:17]([CH2:23][OH:22])[OH:18])=[O:24])[CH2:12][CH2:11][CH2:10][CH2:9]/[CH:8]=[CH:7]\[CH2:6][CH2:5][CH2:4][CH2:3][CH2:2][CH3:1]
# initialization: 
	>> time:  0.016034 sec
24 / 25 atoms were mapped..
complete mappings:  6
correct map:  2
>>> mapping is CORRECT
# 182    reaction
target reaction: [C:6]12=[C:4]([Cl:5])[CH:3]=[C:2]([N:12]=[C:11]1[CH:10]=[CH:9][CH:8]=[CH:7]2)[Cl:1].[C:26]34=[C:24]([Cl:25])[CH:23]=[C:22]([N:32]=[C:31]3[CH:30]=[CH:29][CH:28]=[CH:27]4)[Cl:21].[NH2:13][C@H:14]5[C@H:15]([NH2:16])[CH2:17][CH2:18][CH2:19][CH2:20]5>>[N:12]1=[C:11]2[C:6](=[C:4]([CH:3]=[C

# initialization: 
	>> time:  0.015622 sec
14 / 14 atoms were mapped..
complete mappings:  8
correct map:  0
>>> mapping is CORRECT
# 218    reaction
target reaction: [CH:13]1=[CH:14][CH2:15][CH2:11][CH2:12]1.[O:3]=[C:2]([CH2:4][C:5]=2[CH:6]=[CH:7][CH:8]=[CH:9][CH:10]=2)[Cl:1]>>[CH2:15]1[C@@H:14]2[C@@H:13]([C:2](=[O:3])[C@@H:4]2[C:5]=3[CH:10]=[CH:9][CH:8]=[CH:7][CH:6]=3)[CH2:12][CH2:11]1.[Cl-:1]
# initialization: 
	>> time:  0.01231 sec
14 / 15 atoms were mapped..
complete mappings:  20
correct map:  0
>>> mapping is CORRECT
# 219    reaction
target reaction: [CH2:1]=[C:2]1[C:7](=[CH2:8])[CH:6]=[CH:5][CH:4]=[CH:3]1>>[CH2:1]1[CH2:8][C:7]2=[C:2]1[CH:3]=[CH:4][CH:5]=[CH:6]2
# initialization: 
	>> time:  0.0052 sec
8 / 8 atoms were mapped..
complete mappings:  2
correct map:  0
>>> mapping is CORRECT
# 220    reaction
target reaction: [CH3:11][C:12]([CH3:13])([CH3:14])[O-:15].[CH3:1][CH:2]=[O:3].[CH3:4][CH2:5][O:6][C:7]([CH2:9][Cl:10])=[O:8]>>[CH3:11][C:12]([CH3:13])([CH3:14])[OH:15].[O:8]

# initialization: 
	>> time:  0.004062 sec
29 / 31 atoms were mapped..
complete mappings:  24
correct map:  4
>>> mapping is CORRECT
# 241    reaction
target reaction: [CH:13]=1[CH:12]2[C@@:10]([CH3:11])([C@@:15]([C:17]#[C:18][C:19]=3[CH:20]=[CH:21][CH:22]=[CH:23][CH:24]=3)([OH:16])[CH:14]=1)[CH2:9][CH2:8][CH:7]4[C:6]5=[CH:5][CH:4]=[C:3]([CH:29]=[C:28]5[CH2:27][CH2:26][CH:25]24)[O:2][CH3:1]>>[CH:24]=1[CH:23]=[CH:22][CH:21]=[CH:20][C:19]=1[CH2:18][C:17](=[O:16])[C:15]2=[CH:14][CH:13]=[C:12]3[CH:25]4[CH:7]([C:6]5=[CH:5][CH:4]=[C:3]([CH:29]=[C:28]5[CH2:27][CH2:26]4)[O:2][CH3:1])[CH2:8][CH2:9][C@:10]23[CH3:11]
# initialization: 
	>> time:  0.033657 sec
28 / 29 atoms were mapped..
complete mappings:  2
correct map:  0
>>> mapping is CORRECT


In [504]:
# score = hydrogens + bond
res = dict()
for i, x in enumerate( strns ):
    print( '#', i+1, '   reaction' )
    if i == 39:
        continue
    
    RM = ReactionMap( x )
    idx, maps = dats[i]
    print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
    
    maps_comp = RM.permutation_complete( maps )
    mps_hyd = RM.score( maps_comp, mode='hydrogen' )
    print( 'independent mappings (hydrogen)        :', len( mps_hyd ) )
    mps_hydbn = RM.score( mps_hyd, mode='bond' )
    print( 'independent mappings (hydrogen -> bond):', len( mps_hydbn ) )
    
    gp_sk = RM.isomorphism( mps_hydbn, mode='skelton' )
    print( 'independent mappings (skelton): ', len( gp_sk ) )
    
    gp_bn = RM.isomorphism( mps_hydbn, mode='bond' )
    print( 'independent mappings (bond)   : ', len( gp_bn ) )
    
    res = RM.check_answer( mps_hydbn )
    
    if res != None:
        print( 'correct map: ', mps_hydbn.index( res ) )
        print( '>>> mapping is CORRECT\n')  
    else:
        print( '>>> mapping is WRONG\n')

# 1    reaction
target reaction: [CH2:2]([CH3:3])[SiH:1]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][O:9][C:10]([CH2:11][CH2:12][C:13]([Cl:14])=[O:15])=[O:16]>>[CH2:2]([CH3:3])[Si:1]([CH2:4][CH3:5])([CH2:6][CH3:7])[Cl:14].[CH2:11]([C:10]([O:9][CH3:8])=[O:16])[CH2:12][CH:13]=[O:15]
# initialization: 
	>> time:  0.012718 sec
15 / 16 atoms were mapped..
independent mappings (hydrogen)        : 6
independent mappings (hydrogen -> bond): 6
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 2    reaction
target reaction: [OH:1][OH:2].[OH:3][OH:4].[CH:8]1=[C:7]([O:6][CH3:5])[C:14]([OH:15])=[C:11]([CH:10]=[CH:9]1)[CH:12]=[O:13]>>[OH2:3].[OH2:4].[O:1]=[C:12]=[O:13].[C:7]=1([CH:8]=[CH:9][CH:10]=[C:11]([OH:2])[C:14]=1[OH:15])[O:6][CH3:5]
# initialization: 
	>> time:  0.014661 sec
11 / 15 atoms were mapped..
independent mappings (hydrogen)        : 48
independent mappings (hydrogen -> bond): 48
independent mappings (skelton):  2
independent ma

# initialization: 
	>> time:  0.026502 sec
15 / 16 atoms were mapped..
independent mappings (hydrogen)        : 2
independent mappings (hydrogen -> bond): 2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 21    reaction
target reaction: [CH3:1][CH:2]([CH3:3])[N-:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19]([C@H:8]2[N:12]1[CH2:11][CH2:10][CH2:9]2)=[O:20].[CH:23](=[CH2:24])[CH2:22][Br:21]>>[CH3:1][CH:2]([CH3:3])[NH:4][CH:5]([CH3:6])[CH3:7].[Cl:15][C:14]([Cl:16])([Cl:17])[C@H:13]1[O:18][C:19](=[O:20])[C@@:8]2([CH2:9][CH2:10][CH2:11][N:12]12)[CH2:22][CH:23]=[CH2:24].[Br-:21]
# initialization: 
	>> time:  0.020836 sec
23 / 24 atoms were mapped..
independent mappings (hydrogen)        : 96
independent mappings (hydrogen -> bond): 48
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 22    reaction
target reaction: [Cl:15][C:14]([Cl:16])

independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 35    reaction
target reaction: [CH3:13][C:14]([CH3:29])([CH3:15])[C@@H:16]([CH2:27][OH:28])[NH:17][C:18]([C:19]1=[C:20]([CH:21]=[CH:22][CH:23]=[CH:24]1)[Br:25])=[O:26].[CH3:6][CH2:7][N:8]([CH2:9][CH3:10])[CH2:11][CH3:12].[O:3]=[S:2](=[O:5])([Cl:4])[CH3:1]>>[CH3:13][C:14]([CH3:29])([CH3:15])[C@H:16]1[CH2:27][O:26][C:18]([C:19]=2[CH:24]=[CH:23][CH:22]=[CH:21][C:20]=2[Br:25])=[N:17]1.[CH3:6][CH2:7][NH+:8]([CH2:9][CH3:10])[CH2:11][CH3:12].[O:3]=[S:2](=[O:5])([CH3:1])[OH:28].[Cl-:4]
# initialization: 
	>> time:  0.019736 sec
27 / 29 atoms were mapped..
independent mappings (hydrogen)        : 72
independent mappings (hydrogen -> bond): 72
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 36    reaction
target reaction: [CH2:20]1[CH2:19][CH:18]([CH2:23][CH2:22][CH2:21]1)[N-:24][CH:25]2[CH2:26][CH2:27][CH2:28][CH2:29][CH

# initialization: 
	>> time:  0.005264 sec
8 / 11 atoms were mapped..
independent mappings (hydrogen)        : 4
independent mappings (hydrogen -> bond): 4
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 58    reaction
target reaction: [C-:1]#[O+:2].[C-:3]#[O+:4].[CH2:10]1[CH2:9][N:8]([CH2:34][CH2:33][N:11]1[C:12]([C:13]=2[C:21](=[C:17]([CH:16]=[CH:15][CH:14]=2)[N:18](=[O:19])=[O:20])[CH:22]=[CH:23][C:24]3=[CH:25][C:26](=[CH:28][C:29](=[CH:31]3)[F:30])[Cl:27])=[O:32])[S:6](=[O:35])(=[O:7])[CH3:5]>>[O:2]=[C:1]=[O:19].[O:4]=[C:3]=[O:20].[CH2:10]1[CH2:9][N:8]([CH2:34][CH2:33][N:11]1[C:12]([C:13]=2[C:21]3=[C:17]([CH:16]=[CH:15][CH:14]=2)[NH:18][C:23](=[CH:22]3)[C:24]=4[CH:31]=[C:29]([F:30])[CH:28]=[C:26]([Cl:27])[CH:25]=4)=[O:32])[S:6](=[O:35])(=[O:7])[CH3:5]
# initialization: 
	>> time:  0.01001 sec
33 / 35 atoms were mapped..
independent mappings (hydrogen)        : 64
independent mappings (hydrogen -> bond): 64
independent 

# initialization: 
	>> time:   0.0 sec
7 / 10 atoms were mapped..
independent mappings (hydrogen)        : 2
independent mappings (hydrogen -> bond): 2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 74    reaction
target reaction: [CH3:1][CH2:2][N:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[O:10]=[S:9](=[O:12])([CH3:8])[Cl:11].[CH:15]1=[C:14]([NH2:13])[C:19](=[CH:18][CH:17]=[CH:16]1)[C:20]([CH2:21][C:22]=2[CH:27]=[CH:26][CH:25]=[CH:24][CH:23]=2)=[N:28][OH:29]>>[CH3:1][CH2:2][NH+:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[O:10]=[S:9](=[O:12])([CH3:8])[OH:29].[CH2:21]([C:20]=1[C:19]2=[CH:18][CH:17]=[CH:16][CH:15]=[C:14]2[NH:13][N:28]=1)[C:22]3=[CH:23][CH:24]=[CH:25][CH:26]=[CH:27]3.[Cl-:11]
# initialization: 
	>> time:  0.021751 sec
27 / 29 atoms were mapped..
independent mappings (hydrogen)        : 24
independent mappings (hydrogen -> bond): 12
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping

...

# initialization: 
	>> time:  0.015624 sec
10 / 11 atoms were mapped..
independent mappings (hydrogen)        : 2
independent mappings (hydrogen -> bond): 1
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 165    reaction
target reaction: [O:17]=[S:16](=[O:19])([Cl:18])[C:15]=1[CH:20]=[CH:21][C:12]([CH3:11])=[CH:13][CH:14]=1.[N:9](=[C:2]([C:3]=2[CH:8]=[CH:7][N:6]=[CH:5][CH:4]=2)[CH3:1])[OH:10]>>[O:17]=[S:16](=[O:19])([O:10][N:9]=[C:2]([C:3]1=[CH:4][CH:5]=[N:6][CH:7]=[CH:8]1)[CH3:1])[C:15]2=[CH:20][CH:21]=[C:12]([CH3:11])[CH:13]=[CH:14]2.[ClH:18]
# initialization: 
	>> time:  0.010292 sec
20 / 21 atoms were mapped..
independent mappings (hydrogen)        : 8
independent mappings (hydrogen -> bond): 2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 166    reaction
target reaction: [CH2:2]([CH2:3][CH2:4][CH2:5][CH2:6][CH:7]([CH:8]=[CH2:9])[O:10][C:11](=[O:16])[CH2

# initialization: 
	>> time:  0.018071 sec
24 / 25 atoms were mapped..
independent mappings (hydrogen)        : 4
independent mappings (hydrogen -> bond): 4
independent mappings (skelton):  2
independent mappings (bond)   :  2
>>> mapping is WRONG

# 182    reaction
target reaction: [C:6]12=[C:4]([Cl:5])[CH:3]=[C:2]([N:12]=[C:11]1[CH:10]=[CH:9][CH:8]=[CH:7]2)[Cl:1].[C:26]34=[C:24]([Cl:25])[CH:23]=[C:22]([N:32]=[C:31]3[CH:30]=[CH:29][CH:28]=[CH:27]4)[Cl:21].[NH2:13][C@H:14]5[C@H:15]([NH2:16])[CH2:17][CH2:18][CH2:19][CH2:20]5>>[N:12]1=[C:11]2[C:6](=[C:4]([CH:3]=[C:2]1[NH:16][C@@H:15]3[CH2:17][CH2:18][CH2:19][CH2:20][C@H:14]3[NH:13][C:22]=4[N:32]=[C:31]5[C:26](=[C:24]([CH:23]=4)[Cl:25])[CH:27]=[CH:28][CH:29]=[CH:30]5)[Cl:5])[CH:7]=[CH:8][CH:9]=[CH:10]2.[ClH:1].[ClH:21]
# initialization: 
	>> time:  0.020114 sec
30 / 32 atoms were mapped..
independent mappings (hydrogen)        : 8
independent mappings (hydrogen -> bond): 8
independent mappings (skelton):  1
independent mappings (bond)   :

independent mappings (hydrogen)        : 1024
independent mappings (hydrogen -> bond): 256
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 198    reaction
target reaction: [CH3:14][C:13]1([CH3:15])[O:12][B:11]([O:19][C:16]1([CH3:17])[CH3:18])[C:4]([C:20]=2[CH:25]=[CH:24][CH:23]=[CH:22][CH:21]=2)([C:5]=3[CH:10]=[CH:9][CH:8]=[CH:7][CH:6]=3)[CH2:3][CH:2]=[CH2:1].[OH2:26]>>[CH3:14][C:13]1([CH3:15])[O:12][B:11]([O:19][C:16]1([CH3:17])[CH3:18])[OH:26].[C:5]2(=[CH:6][CH:7]=[CH:8][CH:9]=[CH:10]2)[CH:4]([C:20]3=[CH:21][CH:22]=[CH:23][CH:24]=[CH:25]3)[CH2:3][CH:2]=[CH2:1]
# initialization: 
	>> time:  0.011337 sec
25 / 26 atoms were mapped..
independent mappings (hydrogen)        : 64
independent mappings (hydrogen -> bond): 16
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 199    reaction
target reaction: [CH2:20]1[CH2:21][CH2:22][CH:17]2[CH2:25][CH2:24][CH2:23][CH:19

# initialization: 
	>> time:  0.017323 sec
15 / 16 atoms were mapped..
independent mappings (hydrogen)        : 4
independent mappings (hydrogen -> bond): 4
independent mappings (skelton):  2
independent mappings (bond)   :  2
>>> mapping is WRONG

# 213    reaction
target reaction: [CH:4]1=[CH:5][CH2:6][CH:7]=[CH:8][CH:3]1[CH:2]=[O:1]>>[CH:8]=1[CH2:7][CH2:6][CH:5]=[CH:4][C:3]=1[CH:2]=[O:1]
# initialization: 
	>> time:  0.003349 sec
8 / 8 atoms were mapped..
independent mappings (hydrogen)        : 2
independent mappings (hydrogen -> bond): 2
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 214    reaction
target reaction: [CH3:6][Si:7]([CH3:8])([CH3:9])[O:10][C:11](=[CH2:12])[C:13](=[O:14])[C:15]=1[CH:20]=[CH:19][CH:18]=[CH:17][CH:16]=1.[C:2]([CH:4]=[CH2:5])(=[CH2:3])[CH3:1]>>[CH3:6][Si:7]([CH3:8])([CH3:9])[O:10][C:11]1([CH2:5][CH:4]=[C:2]([CH3:1])[CH2:3][CH2:12]1)[C:13]([C:15]2=[CH:16][CH:17]=[CH:18][CH:19]=[CH:20]2)=[O:1

# initialization: 
	>> time:  0.013069 sec
14 / 18 atoms were mapped..
independent mappings (hydrogen)        : 12
independent mappings (hydrogen -> bond): 12
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 238    reaction
target reaction: [NH:12]([C:10](=[O:11])[CH2:9][CH2:8][C@@H:7](/[CH:6]=[CH:5]/[CH:4]=[CH:3]/[CH2:2][CH3:1])[O:14][CH2:15][C:16]1=[CH:17][CH:18]=[CH:19][CH:20]=[CH:21]1)[OH:13]>>[C:10]1(=[O:11])[N:12]2[O:13][C@@H:3]([CH:4]=[CH:5][C@@H:6]2[C@@H:7]([CH2:8][CH2:9]1)[O:14][CH2:15][C:16]3=[CH:17][CH:18]=[CH:19][CH:20]=[CH:21]3)[CH2:2][CH3:1]
# initialization: 
	>> time:  0.013769 sec
21 / 21 atoms were mapped..
independent mappings (hydrogen)        : 2
independent mappings (hydrogen -> bond): 1
independent mappings (skelton):  1
independent mappings (bond)   :  1
correct map:  0
>>> mapping is CORRECT

# 239    reaction
target reaction: [CH:6]1=[CH:5][CH:4]=[CH:3][C:2](=[CH:7]1)[NH2:1].[O:11]=[CH:10][CH:9]=[C

In [104]:
i = 98 - 1
x = strns[i]
RM = ReactionMap( x )
maps = RM.calculate_mapping()
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
maps_comp = RM.permutation_complete( maps )
mps_hyd = RM.score( maps_comp, mode='hydrogen' )
print( 'independent mappings (hydrogen)        :', len( mps_hyd ) )
mps_hydbn = RM.score( mps_hyd, mode='bond' )
print( 'independent mappings (hydrogen -> bond):', len( mps_hydbn ) )
gp_sk = RM.isomorphism( mps_hydbn, mode='skelton' )
print( 'independent mappings (skelton): ', len( gp_sk ) )
gp_bn = RM.isomorphism( mps_hydbn, mode='bond' )
print( 'independent mappings (bond)   : ', len( gp_bn ) )
res = RM.check_answer( mps_hydbn )
if res != None:
    print( 'correct map: ', mps_hydbn.index( res ) )
    print( '>>> mapping is CORRECT\n')  
else:
    print( '>>> mapping is WRONG\n')

target reaction: [Cl:20][S:21]([Cl:22])=[O:23].[Cl:24][S:25]([Cl:26])=[O:27].[OH:17][C:16](=[O:18])[C:11]1=[C:10]2[O:9][C:8]3=[C:7]([C:15]2=[CH:14][CH:13]=[CH:12]1)[CH:6]=[CH:5][CH:4]=[C:3]3[C:2](=[O:19])[OH:1]>>[O:1]=[S:25]=[O:27].[O:17]=[S:21]=[O:23].[Cl-:22].[Cl-:26].[CH:4]=1[CH:5]=[CH:6][C:7]=2[C:15]=3[C:10]([O:9][C:8]=2[C:3]=1[C:2](=[O:19])[Cl:24])=[C:11]([CH:12]=[CH:13][CH:14]=3)[C:16]([Cl:20])=[O:18]
# initialization: 
	>> time:  0.019194 sec
# find unique pair
	>> 1 pairs were found..
	>> [(9, 9)]
	>> time:  0.042868 sec
# maximum cliques: mode = True , timeout = 10.0
	>>  21 / 27 atoms were mapping..
	>>       241 atoms,    18706 nodes,
	>> time:  10.011092 sec
	>> QUALITY = APPROX
	>> 44 maps were found..
>>>>> mapping time: 10.163701
21 / 27 atoms were mapped..
independent mappings (hydrogen)        : 2112
independent mappings (hydrogen -> bond): 144
independent mappings (skelton):  8
independent mappings (bond)   :  8
correct map:  0
>>> mapping is CORRECT



In [518]:
R = deepcopy( RM.RCT )
R.remap( dict( mps_hydbn[0] ))
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [480]:
i = 19 - 1
x = strns[i]

RM = ReactionMap( x )
idx, maps = dats[i]
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
maps_comp = RM.permutation_complete( maps )
print( 'complete mappings: ', len( maps_comp ) )
mps_sk = RM.isomorphism( maps_comp, mode='skelton' )
print( 'independent mappings (skelton): ', len( mps_sk ) )
mps_bn = RM.isomorphism( maps_comp, mode='bond' )
print( 'independent mappings (bond)   : ', len( mps_bn ) )
selected_mps = [ maps_comp[t] for t in [ x.pop() for x in mps_sk ] ]

res = RM.check_answer( selected_mps )
if res != None:
    print( 'correct map: ', selected_mps.index( res ) )
    print( '>>> mapping is CORRECT' )
else:
    print( '>>> mapping is WRONG' )

target reaction: [CH2:18]1[CH2:17][CH:16]([CH2:21][CH2:20][CH2:19]1)[N-:22][CH:23]2[CH2:24][CH2:25][CH2:26][CH2:27][CH2:28]2.[CH3:5][CH:4]([CH3:6])[C:3]([O:2][CH3:1])=[O:7].[C:9]=3([CH:15]=[CH:14][CH:13]=[C:11]([Br:12])[CH:10]=3)[F:8]>>[CH2:18]1[CH2:17][CH:16]([CH2:21][CH2:20][CH2:19]1)[NH:22][CH:23]2[CH2:24][CH2:25][CH2:26][CH2:27][CH2:28]2.[CH3:5][C:4]([CH3:6])([C:3](=[O:7])[O:2][CH3:1])[C:11]3=[CH:10][C:9](=[CH:15][CH:14]=[CH:13]3)[F:8].[Br-:12]
# initialization: 
	>> time:  0.014613 sec
27 / 28 atoms were mapped..
complete mappings:  32
independent mappings (skelton):  2
independent mappings (bond)   :  2
correct map:  0
>>> mapping is CORRECT


In [481]:
cgr = RM.RCT ^ RM.PRD
cgr.clean2d()
cgr

In [482]:
i = 0
RM.RCT.clean2d()
RM.PRD.clean2d()
R = deepcopy( RM.RCT )
R.remap( dict( selected_mps[i] ))
rxn = ReactionContainer( [R], [RM.PRD] )
print('mapping', i+1)
rxn

mapping 1


In [483]:
CGR = R ^ RM.PRD
CGR.clean2d()
print( 'CGR', i+1 )
CGR

CGR 1


In [445]:
i = 1
R = deepcopy( RM.RCT )
R.remap( dict( selected_mps[i] ))
rxn = ReactionContainer( [R], [RM.PRD] )
print('mapping', i+1)
rxn

mapping 2


In [446]:
CGR = R ^ RM.PRD
CGR.clean2d()
print( 'CGR', i+1 )
CGR

CGR 2


In [415]:
i = 2
R = deepcopy( RM.RCT )
R.remap( dict( selected_mps[i] ))
rxn = ReactionContainer( [R], [RM.PRD] )
print('mapping', i+1)
rxn

IndexError: list index out of range

In [401]:
CGR = R ^ RM.PRD
CGR.clean2d()
print( 'CGR', i+1 )
CGR

CGR 3


In [50]:
i = 6 - 1
x = strns[i]

h_maps = RM.score( maps_comp, mode='hydrogen' )
print( 'Hs min complete mappings: ', len( h_maps ) )
hmps_sk = RM.isomorphism( h_maps, mode='skelton' )
print( 'Hs min independent mappings (skelton): ', len( hmps_sk ) )
hmps_bn = RM.isomorphism( h_maps, mode='bond' )
print( 'Hs min independent mappings (bond)   : ', len( hmps_bn ) )

h_selected_mps = [ h_maps[t] for t in [ x.pop() for x in hmps_bn ] ]
h_res = RM.check_answer( h_selected_mps )

if h_res != None:
    print( 'correct map: ', h_selected_mps.index( h_res ) )
    print( '>>>> mapping is h-CORRECT\n' )
else:
    print( '>>>> mapping is h-WRONG\n' )

NameError: name 'strns' is not defined

In [476]:
i = 0
R = deepcopy( RM.RCT )
mp = h_selected_mps[i]
R.remap( dict( mp ))
cgr = R ^ RM.PRD
cgr.clean2d()
print( sum([ abs(RM.rct.dic_hyd[x] - RM.prd.dic_hyd[y]) for x, y in mp ]) )

cgr

2


In [477]:
i = 1
R = deepcopy( RM.RCT )
mp =  h_selected_mps[i]
R.remap( dict( mp ))
cgr = R ^ RM.PRD
cgr.clean2d()
print( sum([ abs(RM.rct.dic_hyd[x] - RM.prd.dic_hyd[y]) for x, y in mp ]) )
cgr

2


In [464]:
RM.RXN

In [472]:
RM.rct.dic_hyd[18], RM.prd.dic_hyd[18]

(0, 1)

In [451]:
i = 2
R = deepcopy( RM.RCT )
R.remap( dict( h_selected_mps[i] ))
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [453]:
i = 3
R = deepcopy( RM.RCT )
R.remap( dict( h_selected_mps[i] ))
cgr = R ^ RM.PRD
cgr.clean2d()
cgr

In [9]:
with open( 'complex_reaction.csv' ) as f:
     strns = [ x.strip('\n').split(';') for x in f.readlines() ]

In [ ]:
NGs = ['21', '33', '34', '41', '44', '45', '67', '68', '75', '105', '112', '114']
for i, (x, y) in enumerate( strns ):
    if x in NGs:
        continue
    
    print( i, 'Reaction', x, '\n' )
    RM = ReactionMap( y )
    maps = RM.calculate_mapping( timeout=300.0 )
    #gp_sk = RM.isomorphism( maps, mode='skelton' )
    #gp_bn = RM.isomorphism( maps, mode='bond' )
    
    print( 'all mappigns ', len( maps ) )
    #print( 'independent mappigns (skelton)', len( gp_sk ) )
    #print( 'independent mappigns (bond)   ', len( gp_bn ) )
    #selected = [ maps[i] for i in [ x.pop() for x in gp_bn ] ]
    #res = RM.check_answer( selected )
    
    res = RM.check_answer( maps )
    
    if res != None:
        print( '>>> CORRECT\n' )
    else:
        print( '>>> WRONG\n' )

0 Reaction 1 

target reaction: [CH:6]=1[CH:7]=[CH:8][CH:9]=[CH:10][C:5]=1[O:4][CH2:3][CH:2]=[CH2:1]>>[C:5]1([OH:4])=[CH:6][CH:7]=[CH:8][CH:9]=[C:10]1[CH2:1][CH:2]=[CH2:3]
# initialization: 
	>> time:  0.005984 sec
# find unique pair
	>> 0 pairs were found..
	>> []
	>> time:  0.05176 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  10 / 10 atoms were mapping..
	>>        74 atoms,     1329 nodes,
	>> time:   0.0 sec
	>> QUALITY = EXACT
	>> 4 maps were found..
>>>>> mapping time: 0.05176
all mappigns  4
>>> CORRECT

1 Reaction 2 

target reaction: [CH:3](\[CH2:4][CH2:5]/[CH:6]=[CH:7]/[CH3:8])=[CH:2]/[CH3:1]>>[CH:3]([CH:2]([CH:7]([CH:6]=[CH2:5])[CH3:8])[CH3:1])=[CH2:4]
# initialization: 
	>> time:   0.0 sec
# find unique pair
	>> 0 pairs were found..
	>> []
	>> time:  0.043252 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  8 / 8 atoms were mapping..
	>>        49 atoms,      486 nodes,
	>> time:   0.0 sec
	>> QUALITY = EXACT
	>> 8 maps were found..
>>>>> mapping time:

# initialization: 
	>> time:  0.030282 sec
# find unique pair
	>> 1 pairs were found..
	>> [(6, 6)]
	>> time:  0.089841 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  17 / 17 atoms were mapping..
	>>        85 atoms,     2136 nodes,
	>> time:  0.022183 sec
	>> QUALITY = EXACT
	>> 72 maps were found..
>>>>> mapping time: 0.141216
all mappigns  72
>>> CORRECT

13 Reaction 15 

target reaction: [CH3:1][Si:2]([CH3:3])([CH3:4])[CH2:5][CH:6]=[CH2:7].[CH:15]=1[C:10](=[CH:11][CH:12]=[CH:13][CH:14]=1)[CH:9]=[O:8].[OH2:16]>>[CH3:1][Si:2]([CH3:3])([CH3:4])[OH:16].[CH:15]=1[C:10]([CH:9]([OH:8])[CH2:7][CH:6]=[CH2:5])=[CH:11][CH:12]=[CH:13][CH:14]=1
# initialization: 
	>> time:  0.020106 sec
# find unique pair
	>> 1 pairs were found..
	>> [(13, 13)]
	>> time:  0.030158 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  16 / 16 atoms were mapping..
	>>        69 atoms,     1430 nodes,
	>> time:  0.010089 sec
	>> QUALITY = EXACT
	>> 24 maps were found..
>>>>> mapping time: 0.060412
a

# initialization: 
	>> time:  0.022938 sec
# find unique pair
	>> 1 pairs were found..
	>> [(1, 1)]
	>> time:  0.029921 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  14 / 14 atoms were mapping..
	>>        97 atoms,     2584 nodes,
	>> time:   0.0 sec
	>> QUALITY = EXACT
	>> 1 maps were found..
>>>>> mapping time: 0.040989
all mappigns  1
>>> CORRECT

28 Reaction 30 

target reaction: [CH2:18]1[C:16](=[O:17])[CH:15]=[C:14]2[C@:20]([CH:10]3[CH2:9][CH2:8][C@:6]([C:4]([C:2](=[O:3])[CH3:1])=[O:5])([CH:22]([CH:11]3[CH2:12][CH2:13]2)[CH2:23][CH:24]=[O:25])[CH3:7])([CH3:21])[CH2:19]1>>[CH:11]12[CH:10]([CH2:9][CH2:8][C@:6]3([CH3:7])[C@:2]4([OH:3])[C:4](=[O:5])[O:25][C@@H:24]([CH2:1]4)[CH2:23][CH:22]13)[C@@:20]5([C:14]([CH2:13][CH2:12]2)=[CH:15][C:16]([CH2:18][CH2:19]5)=[O:17])[CH3:21]
# initialization: 
	>> time:  0.069676 sec
# find unique pair
	>> 12 pairs were found..
	>> [(9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (17, 17), (18, 18), (19, 19), (20, 20),

# initialization: 
	>> time:  0.050348 sec
# find unique pair
	>> 3 pairs were found..
	>> [(1, 1), (17, 17), (18, 18)]
	>> time:  0.104201 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  18 / 18 atoms were mapping..
	>>       214 atoms,    12696 nodes,
	>> time:  0.445412 sec
	>> QUALITY = EXACT
	>> 1 maps were found..
>>>>> mapping time: 0.768714
all mappigns  1
>>> CORRECT

44 Reaction 47 

target reaction: [C:10]1(=[CH:11][CH:12]=[CH:13][CH:14]=[CH:15]1)[C:9]([C:16]2=[CH:17][CH:18]=[CH:19][CH:20]=[CH:21]2)=[CH:8][C:5]([CH3:6])([CH3:7])[CH:4]=[C:2]([CH3:1])[CH3:3]>>[C:10]1(=[CH:15][CH:14]=[CH:13][CH:12]=[CH:11]1)[C:9]2([C:16]3=[CH:21][CH:20]=[CH:19][CH:18]=[CH:17]3)[C:5]([CH3:6])([CH3:7])[CH:8]2[CH:4]=[C:2]([CH3:1])[CH3:3]
# initialization: 
	>> time:  0.048293 sec
# find unique pair
	>> 0 pairs were found..
	>> []
	>> time:  0.726424 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  21 / 21 atoms were mapping..
	>>       506 atoms,    89736 nodes,
	>> time:  300.0

# maximum cliques: mode = True , timeout = 300.0
	>>  26 / 26 atoms were mapping..
	>>       373 atoms,    49046 nodes,
	>> time:  299.998351 sec
	>> QUALITY = APPROX
	>> 4 maps were found..
>>>>> mapping time: 300.46752
all mappigns  4
>>> CORRECT

54 Reaction 57 

target reaction: [CH:26]1=[C:25]([CH:30]=[CH:29][CH:28]=[CH:27]1)[C:24]2([C:31]3=[CH:32][CH:33]=[CH:34][CH:35]=[CH:36]3)[CH2:23][CH2:22][CH2:21][C@H:20]2[O:19][CH:18]=[CH2:17].[CH3:1][CH:2]([CH3:3])[O:4][C:5](=[O:6])/[CH:7]=[CH:8]/[C:9]([O:11]/[CH:12]=[CH:13]/[N+:14]([O-:15])=[O:16])=[O:10]>>[CH:29]1=[CH:28][CH:27]=[CH:26][C:25](=[CH:30]1)[C:24]2([C:31]3=[CH:36][CH:35]=[CH:34][CH:33]=[CH:32]3)[C@H:20]([O:19][C@H:18]4[O:16][N:14]5[O:15][C@@H:7]([C:5](=[O:6])[O:4][CH:2]([CH3:1])[CH3:3])[C@@H:8]6[C:9](=[O:10])[O:11][C@@H:12]([C@H:13]56)[CH2:17]4)[CH2:21][CH2:22][CH2:23]2
# initialization: 
	>> time:  0.038105 sec
# find unique pair
	>> 5 pairs were found..
	>> [(2, 2), (21, 21), (22, 22), (23, 23), (24, 24)]
	>> time:  0.05212

# maximum cliques: mode = True , timeout = 300.0
	>>  22 / 22 atoms were mapping..
	>>       139 atoms,     5577 nodes,
	>> time:  0.038095 sec
	>> QUALITY = EXACT
	>> 2 maps were found..
>>>>> mapping time: 0.090257
all mappigns  2
>>> CORRECT

70 Reaction 73 

target reaction: [CH3:1][C:2]([CH3:5])([O:3][CH3:4])/[CH:6]=[CH:7]/[C:8]=1[C:15](=[O:16])[C@@H:14]2[C@H:12]([C@@H:10]([CH:9]=1)[OH:11])[O:13]2.[CH3:17][C:18]([CH3:21])([O:19][CH3:20])/[CH:22]=[CH:23]/[C:24]=3[C:31](=[O:32])[C@@H:30]4[C@H:28]([C@@H:26]([CH:25]=3)[OH:27])[O:29]4>>[CH3:1][C:2]([CH3:5])(/[CH:6]=[CH:7]/[C@:8]12[C@H:9]([C@H:25]3[C:24]([C:31]([C@H:30]4[O:29][C@H:28]4[C@@H:26]3[OH:27])=[O:32])=[CH:23][C@@H:22]1[C:18]([CH3:17])([CH3:21])[O:19][CH3:20])[CH:10]([C@@H:12]5[O:13][C@@H:14]5[C:15]2=[O:16])[OH:11])[O:3][CH3:4]
# initialization: 
	>> time:  0.026193 sec
# find unique pair
	>> 0 pairs were found..
	>> []
	>> time:  0.464617 sec
# maximum cliques: mode = True , timeout = 300.0
	>>  32 / 32 atoms were mapping..
	>

In [55]:
i = 112
x, y = strns[i]

print( i, 'Reaction', x, '\n' )
RM = ReactionMap( y )

maps = RM.calculate_mapping( timeout=30.0 )
print( 'all mappigns ', len( maps ) )
res = RM.check_answer( maps )
if res != None:
    print( '>>> CORRECT\n' )
else:
    print( '>>> WRONG\n' )

112 Reaction 116 

target reaction: [CH3:11][C:10]([CH3:12])([CH2:13][CH:14]=[CH2:15])[CH:8]([C:7]#[C:6][C:3]1=[C:2]([CH3:1])[CH2:5][CH2:4]1)[OH:9].[Cl:16][S:17][C:18]2=[CH:19][CH:20]=[CH:21][CH:22]=[CH:23]2>>[CH3:11][C:10]1([CH3:12])[CH2:13][CH:14]2[CH2:15][C:2]3([C:3](=[C:6]([C:7]2=[CH:8]1)[S+:17]([O-:9])[C:18]4=[CH:19][CH:20]=[CH:21][CH:22]=[CH:23]4)[CH2:4][CH2:5]3)[CH3:1].[ClH:16]
# initialization: 
	>> time:  0.01192 sec
# find unique pair
	>> 1 pairs were found..
	>> [(21, 21)]
	>> time:  0.031463 sec
# maximum cliques: mode = True , timeout = 30.0
	>>  23 / 23 atoms were mapping..
	>>       362 atoms,    43824 nodes,
	>> time:  30.000141 sec
	>> QUALITY = APPROX
	>> 2 maps were found..
>>>>> mapping time: 30.390737
all mappigns  2
>>> CORRECT



In [58]:
RM.RXN

In [59]:
cgr = RM.RCT ^ RM.PRD
cgr.clean2d()
cgr

In [71]:
import pandas as pd
df = pd.read_csv( 'partial_mappings_case_40.csv' )

In [77]:
df.n

InvalidIndexError: (slice(None, None, None), slice(None, None, None))

In [1]:
A = [(1,4,['C','C','C','C']),(2,4,['C','C']),(3,4,['C','C']),(4,5,['C','C']),(5,6,['C','O']),(5,7,['C','C','C','C']),(7,8,['C','C']),(7,8,['C','C','C','C']),(8,9,['C','C']),(10,11,['Cl','S']),(11,12,['S','C']),(12,12,['C','C','C','C','C'])]
B = [(1,1,['C','C','C','C','C']),(1,2,['C','S']),(2,3,['S','O']),(2,4,['S','C']),(4,5,['C','C']),(4,9,['C','C']),(5,6,['C','C']),(5,6,['C','C','C','C']),(6,7,['C','C']),(6,8,['C','C','C']),(8,9,['C','C']),(9,10,['C','C']),(10,11,['C','C']),(10,12,['C','C'])]

In [34]:
pair = []
for i,(x,y,z) in enumerate( A ):
    for j,(s,t,u) in enumerate( B ):
        z2 = deepcopy( z )
        z2.reverse()
        if z == u or z2 == u:
            pair.append((i,j))

print( pair )

[(0, 7), (1, 4), (1, 5), (1, 6), (1, 8), (1, 10), (1, 11), (1, 12), (1, 13), (2, 4), (2, 5), (2, 6), (2, 8), (2, 10), (2, 11), (2, 12), (2, 13), (3, 4), (3, 5), (3, 6), (3, 8), (3, 10), (3, 11), (3, 12), (3, 13), (5, 7), (6, 4), (6, 5), (6, 6), (6, 8), (6, 10), (6, 11), (6, 12), (6, 13), (7, 7), (8, 4), (8, 5), (8, 6), (8, 8), (8, 10), (8, 11), (8, 12), (8, 13), (10, 1), (10, 3), (11, 0)]


In [41]:
from itertools import combinations

mod = []
for (v,(i, j)),(w,(s, t)) in combinations( enumerate( pair ), 2 ):
    if i!=s and j!=t:
        A1 = deepcopy( A[i][2] )
        A2 = deepcopy( A[s][2] )
        A_str = []
        if A[i][0] == A[s][0]:
            A1.reverse()
            A_str = A1 + A2            
        if A[i][1] == A[s][0]:
            A_str = A1 + A2
        if A[i][0] == A[s][1]:
            A1.reverse()
            A2.reverse()
            A_str = A1 + A2
        if A[i][1] == A[s][1]:
            A2.reverse()
            A_str = A1 + A2
        
        B1 = deepcopy( B[j][2] )
        B2 = deepcopy( B[t][2] )
        B_str = []
        if B[j][0] == B[t][0]:
            B1.reverse()
            B_str = B1 + B2
        if B[j][1] == B[t][0]:
            B_str = B1 + B2
        if B[j][0] == B[t][1]:
            B1.reverse()
            B2.reverse()
            B_str = B1 + B2
        if B[j][1] == B[t][1]:
            B2.reverse()
            B_str = B1 + B2
        
        if A_str != [] and B_str != []:# and A_str == B_str:
            mod.append((v,w))

print( mod )

[(0, 1), (0, 3), (0, 4), (1, 10), (1, 11), (1, 18), (1, 19), (2, 9), (2, 13), (2, 14), (2, 17), (2, 21), (2, 22), (3, 9), (3, 12), (3, 17), (3, 20), (4, 11), (4, 19), (5, 10), (5, 14), (5, 18), (5, 22), (6, 10), (6, 13), (6, 15), (6, 16), (6, 18), (6, 21), (6, 23), (6, 24), (7, 14), (7, 16), (7, 22), (7, 24), (8, 14), (8, 15), (8, 22), (8, 23), (9, 18), (9, 19), (10, 17), (10, 21), (10, 22), (11, 17), (11, 20), (12, 19), (13, 18), (13, 22), (14, 18), (14, 21), (14, 23), (14, 24), (15, 22), (15, 24), (16, 22), (16, 23), (17, 25), (19, 25), (20, 25), (25, 26), (25, 28), (25, 29), (26, 34), (26, 36), (26, 37), (27, 35), (27, 39), (27, 40), (28, 34), (28, 35), (28, 38), (29, 34), (29, 37), (30, 36), (30, 40), (31, 36), (31, 39), (31, 41), (31, 42), (32, 40), (32, 42), (33, 40), (33, 41), (34, 35), (34, 37), (34, 38), (43, 45)]


In [43]:
A[1], A[11], B[11], B[1]

((2, 4, ['C', 'C']),
 (12, 12, ['C', 'C', 'C', 'C', 'C']),
 (1, 1, ['C', 'C', 'C', 'C', 'C']),
 (1, 2, ['C', 'S']))

In [42]:
cqs = largest_cliques_limited( mod, 10.0 )
maps = []
for cq in cqs:
    mp = []
    for x in cq:
        mp.append( pair[x] )
    maps.append( mp )
maps

[[(1, 11), (2, 13), (3, 12)],
 [(1, 13), (2, 11), (3, 12)],
 [(1, 12), (2, 13), (3, 11)],
 [(1, 5), (2, 10), (3, 11)],
 [(1, 10), (2, 5), (3, 11)],
 [(1, 13), (2, 12), (3, 11)],
 [(1, 5), (2, 11), (3, 10)],
 [(1, 10), (2, 11), (3, 5)],
 [(1, 12), (2, 11), (3, 13)],
 [(1, 11), (2, 12), (3, 13)],
 [(1, 11), (2, 5), (3, 10)],
 [(1, 11), (2, 10), (3, 5)],
 [(6, 4), (7, 7), (8, 6)],
 [(6, 8), (7, 7), (8, 6)],
 [(6, 6), (7, 7), (8, 4)],
 [(6, 6), (7, 7), (8, 8)]]

In [49]:
A[6], B[4]

((7, 8, ['C', 'C']), (4, 5, ['C', 'C']))

In [48]:
A[7], B[7]

((7, 8, ['C', 'C', 'C', 'C']), (5, 6, ['C', 'C', 'C', 'C']))

In [47]:
A[3], B[12]

((4, 5, ['C', 'C']), (10, 11, ['C', 'C']))

In [33]:
s = [1,2,3]
t = deepcopy( s )
t.reverse()
s, t

([1, 2, 3], [3, 2, 1])

In [93]:
a = '[(1, 12), (2, 11), (3, 10), (4, 9), (5, 8), (6, 13), (7, 14), (8, 1), (9, 2), (10, 3), (11, 6), (12, 7), (13, 4), (14, 5), (15, 27), (16, 22), (17, 23), (18, 26), (19, 24), (20, 25), (21, 21), (22, 16), (23, 17), (24, 18), (25, 20), (26, 19), (27, 15), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 39), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 14), (2, 13), (3, 10), (4, 11), (5, 12), (6, 9), (7, 8), (8, 1), (9, 2), (10, 3), (11, 6), (12, 7), (13, 4), (14, 5), (15, 27), (16, 22), (17, 23), (18, 26), (19, 24), (20, 25), (21, 15), (22, 16), (23, 17), (24, 20), (25, 18), (26, 19), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 39), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 5), (2, 4), (3, 3), (4, 6), (5, 7), (6, 2), (7, 1), (8, 14), (9, 13), (10, 10), (11, 9), (12, 8), (13, 11), (14, 12), (15, 27), (16, 22), (17, 23), (18, 25), (19, 26), (20, 24), (21, 21), (22, 16), (23, 17), (24, 19), (25, 18), (26, 20), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 39), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 8), (2, 9), (3, 10), (4, 13), (5, 14), (6, 11), (7, 12), (8, 5), (9, 4), (10, 3), (11, 6), (12, 7), (13, 2), (14, 1), (15, 15), (16, 16), (17, 17), (18, 19), (19, 20), (20, 18), (21, 27), (22, 22), (23, 23), (24, 25), (25, 24), (26, 26), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 41), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 6), (5, 7), (6, 4), (7, 5), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 19), (19, 20), (20, 18), (21, 28), (22, 22), (23, 23), (24, 24), (25, 26), (26, 25), (27, 27), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 41), (41, 40), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 8), (2, 9), (3, 10), (4, 13), (5, 14), (6, 11), (7, 12), (8, 7), (9, 6), (10, 3), (11, 4), (12, 5), (13, 2), (14, 1), (15, 27), (16, 22), (17, 23), (18, 25), (19, 26), (20, 24), (21, 21), (22, 16), (23, 17), (24, 20), (25, 19), (26, 18), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 14), (2, 13), (3, 10), (4, 11), (5, 12), (6, 9), (7, 8), (8, 5), (9, 4), (10, 3), (11, 2), (12, 1), (13, 6), (14, 7), (15, 21), (16, 22), (17, 23), (18, 26), (19, 25), (20, 24), (21, 28), (22, 16), (23, 17), (24, 19), (25, 20), (26, 18), (27, 15), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 6), (5, 7), (6, 4), (7, 5), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 21), (16, 22), (17, 23), (18, 25), (19, 24), (20, 26), (21, 15), (22, 16), (23, 17), (24, 20), (25, 19), (26, 18), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 41), (41, 40), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 12), (2, 11), (3, 10), (4, 9), (5, 8), (6, 13), (7, 14), (8, 5), (9, 4), (10, 3), (11, 2), (12, 1), (13, 6), (14, 7), (15, 28), (16, 16), (17, 17), (18, 19), (19, 18), (20, 20), (21, 15), (22, 22), (23, 23), (24, 25), (25, 24), (26, 26), (27, 27), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 41), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 12), (2, 11), (3, 10), (4, 9), (5, 8), (6, 13), (7, 14), (8, 5), (9, 4), (10, 3), (11, 6), (12, 7), (13, 2), (14, 1), (15, 15), (16, 16), (17, 17), (18, 20), (19, 18), (20, 19), (21, 27), (22, 22), (23, 23), (24, 26), (25, 24), (26, 25), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 14), (2, 13), (3, 10), (4, 9), (5, 8), (6, 11), (7, 12), (8, 5), (9, 4), (10, 3), (11, 6), (12, 7), (13, 2), (14, 1), (15, 21), (16, 22), (17, 23), (18, 25), (19, 24), (20, 26), (21, 27), (22, 16), (23, 17), (24, 19), (25, 18), (26, 20), (27, 15), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 39), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 14), (9, 13), (10, 10), (11, 9), (12, 8), (13, 11), (14, 12), (15, 15), (16, 16), (17, 17), (18, 19), (19, 18), (20, 20), (21, 28), (22, 22), (23, 23), (24, 25), (25, 24), (26, 26), (27, 27), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 5), (2, 4), (3, 3), (4, 2), (5, 1), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 13), (12, 14), (13, 11), (14, 12), (15, 15), (16, 16), (17, 17), (18, 20), (19, 18), (20, 19), (21, 28), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 14), (9, 13), (10, 10), (11, 9), (12, 8), (13, 11), (14, 12), (15, 21), (16, 22), (17, 23), (18, 26), (19, 25), (20, 24), (21, 15), (22, 16), (23, 17), (24, 19), (25, 20), (26, 18), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 41), (41, 40), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 27), (16, 22), (17, 23), (18, 25), (19, 24), (20, 26), (21, 15), (22, 16), (23, 17), (24, 18), (25, 19), (26, 20), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 28), (16, 16), (17, 17), (18, 19), (19, 18), (20, 20), (21, 15), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 41), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 14), (2, 13), (3, 10), (4, 9), (5, 8), (6, 11), (7, 12), (8, 1), (9, 2), (10, 3), (11, 6), (12, 7), (13, 4), (14, 5), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 27), (22, 22), (23, 23), (24, 26), (25, 25), (26, 24), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 41), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 6), (5, 7), (6, 4), (7, 5), (8, 14), (9, 13), (10, 10), (11, 9), (12, 8), (13, 11), (14, 12), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 26), (25, 24), (26, 25), (27, 27), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 39), (41, 40), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 14), (2, 13), (3, 10), (4, 11), (5, 12), (6, 9), (7, 8), (8, 1), (9, 2), (10, 3), (11, 6), (12, 7), (13, 4), (14, 5), (15, 15), (16, 16), (17, 17), (18, 20), (19, 18), (20, 19), (21, 28), (22, 22), (23, 23), (24, 26), (25, 24), (26, 25), (27, 27), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 12), (2, 11), (3, 10), (4, 13), (5, 14), (6, 9), (7, 8), (8, 1), (9, 2), (10, 3), (11, 4), (12, 5), (13, 6), (14, 7), (15, 21), (16, 22), (17, 23), (18, 25), (19, 26), (20, 24), (21, 27), (22, 16), (23, 17), (24, 18), (25, 20), (26, 19), (27, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 40), (40, 39), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 12), (2, 11), (3, 10), (4, 13), (5, 14), (6, 9), (7, 8), (8, 1), (9, 2), (10, 3), (11, 6), (12, 7), (13, 4), (14, 5), (15, 15), (16, 16), (17, 17), (18, 19), (19, 18), (20, 20), (21, 27), (22, 22), (23, 23), (24, 25), (25, 26), (26, 24), (27, 21), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 41), (40, 40), (41, 39), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)], [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 13), (12, 14), (13, 11), (14, 12), (15, 21), (16, 22), (17, 23), (18, 24), (19, 25), (20, 26), (21, 27), (22, 16), (23, 17), (24, 18), (25, 20), (26, 19), (27, 15), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48)]'
partial_maps = list( ast.literal_eval( a ) )
partial_maps

[[(1, 12),
  (2, 11),
  (3, 10),
  (4, 9),
  (5, 8),
  (6, 13),
  (7, 14),
  (8, 1),
  (9, 2),
  (10, 3),
  (11, 6),
  (12, 7),
  (13, 4),
  (14, 5),
  (15, 27),
  (16, 22),
  (17, 23),
  (18, 26),
  (19, 24),
  (20, 25),
  (21, 21),
  (22, 16),
  (23, 17),
  (24, 18),
  (25, 20),
  (26, 19),
  (27, 15),
  (29, 29),
  (30, 30),
  (31, 31),
  (32, 32),
  (33, 33),
  (34, 34),
  (35, 35),
  (36, 36),
  (37, 37),
  (38, 38),
  (39, 40),
  (40, 39),
  (41, 41),
  (42, 42),
  (43, 43),
  (44, 44),
  (45, 45),
  (46, 46),
  (47, 47),
  (48, 48)],
 [(1, 14),
  (2, 13),
  (3, 10),
  (4, 11),
  (5, 12),
  (6, 9),
  (7, 8),
  (8, 1),
  (9, 2),
  (10, 3),
  (11, 6),
  (12, 7),
  (13, 4),
  (14, 5),
  (15, 27),
  (16, 22),
  (17, 23),
  (18, 26),
  (19, 24),
  (20, 25),
  (21, 15),
  (22, 16),
  (23, 17),
  (24, 20),
  (25, 18),
  (26, 19),
  (27, 28),
  (29, 29),
  (30, 30),
  (31, 31),
  (32, 32),
  (33, 33),
  (34, 34),
  (35, 35),
  (36, 36),
  (37, 37),
  (38, 38),
  (39, 40),
  (40, 39),
  (

In [94]:
i = 40 - 1
x = strns[i]
RM = ReactionMap( x )
print( len( maps[0] ), '/', len( RM.rct.atoms ), 'atoms were mapped..' )
maps_comp = RM.permutation_complete( partial_maps )
print( 'complete mappings: ', len( maps_comp ) )
mps_sk = RM.isomorphism( maps_comp, mode='skelton' )
print( 'independent mappings (skelton): ', len( mps_sk ) )
mps_bn = RM.isomorphism( maps_comp, mode='bond' )
print( 'independent mappings (bond)   : ', len( mps_bn ) )
selected_mps = [ maps_comp[t] for t in [ x.pop() for x in mps_sk ] ]

res = RM.check_answer( selected_mps )
if res != None:
    print( 'correct map: ', selected_mps.index( res ) )
    print( '>>> mapping is CORRECT' )
else:
    print( '>>> mapping is WRONG' )

target reaction: [CH3:1][CH2:2][N:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH3:8][CH2:9][N:10]([CH2:11][CH3:12])[CH2:13][CH3:14].[F:18][C:17]([F:19])([F:20])[C:16](=[O:15])[O:21][C:22]([C:23]([F:24])([F:25])[F:26])=[O:27].[F:39][C:38]([F:40])([F:41])[C:35]=1[CH:36]=[N:37][C:32](=[CH:33][CH:34]=1)[CH2:31][C:30]([C:42]=2[CH:48]=[CH:47][CH:46]=[C:44]([Br:45])[CH:43]=2)=[N:29][OH:28]>>[CH2:2]([CH3:1])[NH+:3]([CH2:4][CH3:5])[CH2:6][CH3:7].[CH2:9]([CH3:8])[NH+:10]([CH2:11][CH3:12])[CH2:13][CH3:14].[F:18][C:17]([F:19])([F:20])[C:16]([O-:28])=[O:15].[F:24][C:23]([F:25])([F:26])[C:22]([O-:21])=[O:27].[F:39][C:38]([F:40])([F:41])[C:35]=1[CH:36]=[N:37][C:32]([CH:31]2[C:30]([C:42]3=[CH:43][C:44]([Br:45])=[CH:46][CH:47]=[CH:48]3)=[N:29]2)=[CH:33][CH:34]=1
# initialization: 
	>> time:  0.026144 sec
10 / 48 atoms were mapped..
complete mappings:  22
independent mappings (skelton):  1
independent mappings (bond)   :  4
correct map:  0
>>> mapping is CORRECT
